In [14]:
# %% [markdown]
# # Dynamic Stock Financial Analysis & DCF Valuation
# **(Interactive Report - Dynamic/User Input Driven Assumptions - V21.7.3 - Ratio safe_div Fix)**
#
# **Analyst:** Simulated AI Research Analyst
#
# **Objective:** To perform a comprehensive financial analysis and estimate the intrinsic value per share using DCF. User selects FCFF method from 5 calculated historical options. **Uses consistent Lakh/Crore formatting.** Initial growth: GM of Rev/FCFF CAGR.
#
# **Methodology:**
# 1.  User Input & Dynamic Fetching.
# 2.  Data Acquisition.
# 3.  WACC Calculation.
# 4.  FCFF Method Pre-Calculation & User Selection (5 Methods Calculated, Displays Avg FCFF).
# 5.  FCFF Calculation & Projection (3 Years, Selected Method Only, Halts if basis non-positive, TV Syntax Fixed).
# 6.  Terminal Value.
# 7.  Discounting & Valuation (Bridge Syntax Fixed).
# 8.  Ratio Analysis (**safe_div Syntax Fixed**).
# 9.  Interactive Visualization.
# 10. **Consistent Indian Number Formatting (Lakh/Crore) enforced.**

# %% [markdown]
# ## 1. Setup and Libraries
# (Section Unchanged)

# %%
# Standard Libraries
import warnings
import sys
import numpy as np
import pandas as pd
import datetime
import math # For isnan, isinf checks
import traceback # For detailed error logging

# Data Fetching
import yfinance as yf

# Visualization (Using Plotly)
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Display Utilities
from IPython.display import display, Markdown, HTML

# Settings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pio.templates.default = "plotly_white"

# %% [markdown]
# ## 2. Configuration and Helper Functions (FCFF Methods Updated V21.7)

# %%
# --- Model Configuration ---
DEFAULT_TICKER = "MUTHOOTFIN.NS"
FORECAST_YEARS = 3
TERMINAL_GROWTH_RATE_CAP = 0.05
DEFAULT_GROWTH_RATE = 0.065
DEFAULT_RF_RATE_SUGGESTION = 7.0
DEFAULT_CURRENCY_SYMBOL = "₹" # Set to Rupee by default
DEFAULT_TAX_RATE = 0.25
DEFAULT_CREDIT_SPREAD = 0.015
DEFAULT_MARKET_RISK_PREMIUM = 0.06

# --- Generic Fallback Benchmark Ratios ---
DEFAULT_BENCHMARK_RATIOS = {
    'P/E': 17.3, 'P/B': 2.5, 'EV/EBITDA': 11.0, 'ROE': 19.05,
    'ROA': 3.92, 'ROCE': 11.85,'Asset Turnover': 0.15,
    'Current Ratio': 1.5, 'Quick Ratio': 1.5
}

# --- Model Stability Bounds ---
MIN_WACC = 0.06; MAX_WACC = 0.25; MIN_TAX_RATE = 0.10; MAX_TAX_RATE = 0.40
MIN_GROWTH = 0.00; MIN_INITIAL_GROWTH_FLOOR = 0.011; MAX_INITIAL_GROWTH_CEILING = 0.15
MIN_COST_OF_DEBT = 0.03; MAX_COST_OF_DEBT = 0.18

# --- Ratio Keys Needed for Benchmarking ---
REQUIRED_BENCHMARK_RATIOS = list(DEFAULT_BENCHMARK_RATIOS.keys())

# --- FCFF Method Definitions (V21.7 - Updated based on user request) ---
# Note: Method 6 (UNI) was identical to Method 4 and omitted.
# Note: Method 7 (Forecast) cannot be used for historical avg.
FCFF_METHODS_DEFINITION = {
    0: {"name": "Operating Profit Approach (EBIT-Based)",
        "formula": "EBIT*(1-Tax Rate) + D&A - ΔWC - CapEx"},
    1: {"name": "NOPAT Approach",
        "formula": "NOPAT + D&A - ΔWC - CapEx [NOPAT = EBIT*(1-T)]"}, # Same calc as 0
    2: {"name": "Cash Flow Statement Adjustment Method (CFO-Based)",
        "formula": "CFO + Interest*(1-Tax Rate) - CapEx"},
    3: {"name": "Bottom-Up Approach (Net Income-Based)",
        "formula": "Net Income + Interest*(1-T) + D&A - ΔWC - CapEx"},
    4: {"name": "EBITDA Proxy Method",
        "formula": "EBITDA - Tax Provision - ΔWC - CapEx"} # Using Tax Provision as proxy for Taxes Paid
}


# --- Helper Functions ---
def print_keys(statement, name): # (Unchanged)
    print(f"\n--- Keys in {name} ---");
    if isinstance(statement, pd.DataFrame): print(", ".join(map(str, [idx for idx in statement.index if idx is not None])))
    elif isinstance(statement, dict): print(", ".join(statement.keys()))
    print("--------------------\n")

# Corrected try/except structure and conversion logic
def safe_get(data, key_list, statement_name="Statement", default=np.nan):
    """Safely retrieves and attempts to convert financial data to float."""
    if not isinstance(key_list, list): key_list = [key_list]
    value = None
    found_key = None

    for key in key_list:
        current_value = None
        raw_val = None
        data_source = None # Keep track of where we might find it

        try: # Outer try: Getting the raw value
            if isinstance(data, pd.Series):
                data_source = f"'{statement_name}' Series"
                if key in data.index:
                    raw_val = data.loc[key]
            elif isinstance(data, dict):
                data_source = f"'{statement_name}' Dict"
                raw_val = data.get(key)
            else:
                pass # Raw_val remains None

            # Inner processing: Converting raw_val if found
            if raw_val is not None:
                if isinstance(raw_val, str) and raw_val.lower() in ['n/a', 'none', '', '--', '-']:
                     pass
                elif isinstance(raw_val, (int, float)):
                    if np.isfinite(raw_val):
                        current_value = float(raw_val)
                elif isinstance(raw_val, pd.Series):
                     if not raw_val.empty:
                         series_val = raw_val.iloc[0]
                         if isinstance(series_val, (int, float)) and np.isfinite(series_val):
                             current_value = float(series_val)
                         elif pd.notna(series_val):
                             try:
                                 temp_val = float(series_val)
                                 if np.isfinite(temp_val):
                                     current_value = temp_val
                             except (ValueError, TypeError): pass
                else:
                    try:
                        temp_val = float(raw_val)
                        if np.isfinite(temp_val):
                            current_value = temp_val
                    except (ValueError, TypeError): pass

        except Exception as e: pass # Let current_value remain None

        if current_value is not None:
            value = current_value
            found_key = key
            break # Exit the loop once a valid value is found

    if value is None: value = default
    return value

# MODIFIED format_currency to ALWAYS use Indian Lakh/Crore format
def format_currency(value, symbol="₹", precision=2):
    """Formats currency value using Indian Lakh/Crore notation ONLY."""
    symbol = "₹"
    if value is None or pd.isna(value) or not math.isfinite(value): return "N/A"
    if abs(value) > 1e18: return "Inf"
    sign = "-" if value < 0 else ""; abs_val = abs(value)

    if abs_val >= 1e7: formatted_val = f"{abs_val / 1e7:.{precision}f} Cr"
    elif abs_val >= 1e5: formatted_val = f"{abs_val / 1e5:.{precision}f} Lakh"
    else: formatted_val = f"{abs_val:,.{precision}f}"
    return f"{sign}{symbol} {formatted_val}"

def format_percent(value, precision=1): # (Unchanged)
     if value is None or pd.isna(value) or not math.isfinite(value): return "N/A"
     if abs(value) > 1e5: return "Inf"
     return f"{value * 100:.{precision}f}%"

def get_validated_float_input(prompt_message, default_value=None, allow_blank=False, min_val=None, max_val=None, is_percentage=False): # (Unchanged)
    while True:
        prompt_with_default = prompt_message
        if is_percentage: prompt_with_default += " (e.g., '15' for 15%)"
        if default_value is not None: prompt_with_default += f" [Default: {default_value}]: "
        elif allow_blank: prompt_with_default += f" [Leave blank]: "
        else: prompt_with_default += ": "
        try:
            user_input_str = input(prompt_with_default).strip()
            if not user_input_str:
                if allow_blank: return None
                if default_value is not None: return float(default_value)
                raise ValueError("Input cannot be empty.")
            if user_input_str.endswith('%'): user_input_str = user_input_str[:-1]
            value = float(user_input_str)
            if min_val is not None and value < min_val: raise ValueError(f"Value must be >= {min_val}")
            if max_val is not None and value > max_val: raise ValueError(f"Value must be <= {max_val}")
            return value
        except ValueError as e: print(f"❌ Invalid input: {e}. Please enter a valid number within the allowed range.")

def try_get_dynamic_rf(rf_ticker_symbol="^TNX"): # (Unchanged)
    print(f"Attempting fetch: Rf ({rf_ticker_symbol})...")
    try:
        rf_ticker=yf.Ticker(rf_ticker_symbol); hist=rf_ticker.history(period="5d")
        if not hist.empty and 'Close' in hist.columns:
            last_close=hist['Close'].iloc[-1]
            if pd.notna(last_close):
                dynamic_rf=last_close/100.0
                if 0.001 < dynamic_rf < 0.20:
                    print(f"✅ Dynamic Rf fetched: {format_percent(dynamic_rf)}")
                    return dynamic_rf
                else:
                    print(f"⚠️ Dynamic Rf ({format_percent(dynamic_rf)}) out of expected range [0.1%, 20%]. Ignoring.")
                    return None
            else: print(f"⚠️ Rf data contains NaN."); return None
        else: print(f"⚠️ No recent history data for Rf ticker."); return None
    except Exception as e: print(f"❌ Error fetching Rf rate for {rf_ticker_symbol}: {e}"); return None

# --- Growth Calculation Helpers (Unchanged) ---
def calculate_cagr_modified(start_val, end_val, num_years): # (Unchanged)
    if pd.isna(start_val) or pd.isna(end_val) or not math.isfinite(start_val) or not math.isfinite(end_val) or num_years <= 0: return np.nan
    if start_val <= 1e-9: return np.nan
    cagr = np.nan
    try:
        if end_val <= 1e-9: return np.nan
        else:
            ratio = end_val / start_val
            if ratio > 0: cagr = ratio**(1 / num_years) - 1
            else: return np.nan
    except Exception as e: cagr = np.nan
    if pd.notna(cagr) and not (-0.999 < cagr < 20.0): cagr = np.nan
    return cagr

def geometric_mean_modified(rates): # (Unchanged)
    valid_rates = [r for r in rates if pd.notna(r) and math.isfinite(r)]
    if not valid_rates: return np.nan
    if len(valid_rates) == 1: return valid_rates[0]
    offset = 1.0
    factors_offset = [r + offset for r in valid_rates]
    if any(f <= 1e-9 for f in factors_offset):
        print("⚠️ GM calculation failure (rate <= -100%). Using arithmetic mean as fallback.")
        return np.mean(valid_rates)
    n = len(valid_rates)
    product = 1.0
    try:
        for factor in factors_offset: product *= factor
        if product < 0 and n % 2 == 0:
            print("⚠️ GM calculation failure (negative base with even root). Using arithmetic mean as fallback.")
            return np.mean(valid_rates)
        gm_offset = np.sign(product) * (np.abs(product)**(1 / n))
        gm = gm_offset - offset
        if pd.notna(gm) and not (-0.999 < gm < 20.0):
            print(f"⚠️ GM result ({format_percent(gm)}) out of bounds [-99.9%, +2000%]. Using arithmetic mean.")
            return np.mean(valid_rates)
        return gm
    except Exception as e:
        print(f"⚠️ GM numeric error ({e}). Using arithmetic mean as fallback.")
        return np.mean(valid_rates)


# --- FCFF Component Fetcher (Added Net Income keys) ---
def _get_fcff_components(fin_col, bs_col, cf_col, bs_prev_col, log_list):
    components = {}; log_list.append("--- Fetching FCFF Components ---")
    rev_keys=["Total Revenue","Revenue"]; components['revenue']=safe_get(fin_col, rev_keys, "IS", default=np.nan); log_list.append(f"    Revenue: {format_currency(components['revenue'])}")
    ebit_keys=["EBIT","Ebit","Operating Income"]; components['ebit']=safe_get(fin_col, ebit_keys, "IS", default=np.nan)
    if not np.isfinite(components['ebit']):
        pretax=safe_get(fin_col, ['Pretax Income'], "IS(Fallback)", default=np.nan); int_exp=abs(safe_get(fin_col, ['Interest Expense', 'Interest Expense Non Operating'], "IS(Fallback)", default=np.nan))
        if np.isfinite(pretax) and np.isfinite(int_exp): components['ebit']=pretax+int_exp; log_list.append(f"    EBIT (Derived): {format_currency(components['ebit'])}")
        else: log_list.append(f"    EBIT: N/A"); components['ebit']=np.nan
    else: log_list.append(f"    EBIT: {format_currency(components['ebit'])}")

    dep_keys=["Depreciation And Amortization","Depreciation"]; components['dep_amort']=safe_get(cf_col, dep_keys, "CF", default=np.nan)
    if not np.isfinite(components['dep_amort']):
        components['dep_amort']=safe_get(fin_col, dep_keys, "IS(Fallback)", default=np.nan)
        if np.isfinite(components['dep_amort']): log_list.append(f"    Dep&Amort (from IS): {format_currency(components['dep_amort'])}")
        else: log_list.append(f"    Dep&Amort: N/A")
    else: log_list.append(f"    Dep&Amort (from CF): {format_currency(components['dep_amort'])}")

    if np.isfinite(components.get('ebit')) and np.isfinite(components.get('dep_amort')):
        components['ebitda']=components['ebit']+components['dep_amort']; log_list.append(f"    EBITDA (Calculated): {format_currency(components['ebitda'])}")
    else: components['ebitda']=np.nan; log_list.append(f"    EBITDA: N/A")

    capex_keys=["Capital Expenditure","Net Capital Expenditure", "Purchase Of Property Plant Equipment"]; capex_val=safe_get(cf_col, capex_keys, "CF", default=np.nan); components['capex']=abs(capex_val) if np.isfinite(capex_val) else np.nan; log_list.append(f"    CapEx (Abs): {format_currency(components['capex'])}")

    wc_keys=["Change In Working Capital", "Changes In Working Capital"]; components['wc_cf']=safe_get(cf_col, wc_keys, "CF(WC)", default=np.nan)
    if np.isfinite(components['wc_cf']):
        components['wc']=components['wc_cf']; components['wc_src']="CF Direct"; log_list.append(f"    WC Change (from CF Direct): {format_currency(components['wc'])}")
    else:
        components['wc_src']="BS Calc Attempt"; components['wc']=np.nan
        log_list.append("    WC Change (CF Direct not found, trying BS calc...)")
        if isinstance(bs_prev_col, pd.Series) and isinstance(bs_col, pd.Series):
            curr_a=safe_get(bs_col, ['Current Assets', 'Total Current Assets'], 'BS Current',np.nan)
            curr_l=safe_get(bs_col, ['Current Liabilities', 'Total Current Liabilities'], 'BS Current',np.nan)
            prev_a=safe_get(bs_prev_col, ['Current Assets', 'Total Current Assets'], 'BS Previous',np.nan)
            prev_l=safe_get(bs_prev_col, ['Current Liabilities', 'Total Current Liabilities'], 'BS Previous',np.nan)
            if all(map(np.isfinite, [curr_a, curr_l, prev_a, prev_l])):
                current_nwc = curr_a - curr_l; previous_nwc = prev_a - prev_l
                components['wc'] = current_nwc - previous_nwc
                components['wc_src']="BS Calc Success"
                log_list.append(f"    WC Change (BS Calculated): {format_currency(components['wc'])}")
            else: components['wc_src']="BS Calc Fail"; log_list.append("    WC Change (BS Calc): Failed")
        else: components['wc_src']="No Prev BS"; log_list.append("    WC Change (BS Calc): Failed")

    cfo_keys=["Cash Flow From Operating Activities", "Operating Cash Flow"]; components['cfo']=safe_get(cf_col, cfo_keys, "CF", default=np.nan); log_list.append(f"    CFO: {format_currency(components['cfo'])}")
    int_exp_keys=['Interest Expense', 'Interest Expense Non Operating']; components['int_exp']=abs(safe_get(fin_col, int_exp_keys, "IS", default=np.nan)); log_list.append(f"    Interest Exp (Abs): {format_currency(components['int_exp'])}")
    tax_keys=['Tax Provision', 'Income Tax Paid', 'Income Tax Expense']; components['tax_prov']=safe_get(fin_col, tax_keys, "IS", default=np.nan)
    if not np.isfinite(components['tax_prov']):
        components['tax_prov']=safe_get(cf_col, tax_keys, "CF(Fallback)", default=np.nan)
        if np.isfinite(components['tax_prov']): log_list.append(f"    Tax Prov (from CF): {format_currency(components['tax_prov'])}")
        else: log_list.append(f"    Tax Prov: N/A")
    else: log_list.append(f"    Tax Prov (from IS): {format_currency(components['tax_prov'])}")

    # --> Added for Net Income-Based Method <--
    ni_keys = ["Net Income", "Net Income Common Stockholders", "Net Income Continuous Operations"]
    components['net_income'] = safe_get(fin_col, ni_keys, "IS", default=np.nan)
    log_list.append(f"    Net Income: {format_currency(components['net_income'])}")

    return components

# --- FCFF Calculation Helper: Single Method (Updated for 5 new methods) ---
def calculate_fcff_single_method_single_year(method_choice, fin_col, bs_col, cf_col, bs_prev_col, tax_rt, log_list_parent):
    log_list_local=[]; c=_get_fcff_components(fin_col, bs_col, cf_col, bs_prev_col, log_list_local); log_list_parent.extend(log_list_local)
    fcff_val=np.nan
    method_info = FCFF_METHODS_DEFINITION.get(method_choice, {})
    method_name = method_info.get("name", "?")
    log_list_parent.append(f"--- Calc FCFF: {method_name} (Method {method_choice}) ---")

    valid_tax = np.isfinite(tax_rt)
    # Check if tax rate is required and valid for the chosen method
    if not valid_tax and method_choice in [0, 1, 2, 3]: # Methods needing tax rate
        log_list_parent.append(f"    ❌ Invalid Tax Rate ({tax_rt}) needed for method {method_choice}. Cannot calculate.")
        return np.nan

    # Check for required components based on method
    required_comps = {}
    missing_comps_list = []

    try:
        if method_choice == 0: # Operating Profit Approach (EBIT-Based)
            required_comps = {'EBIT': c.get('ebit'), 'D&A': c.get('dep_amort'), 'WC': c.get('wc'), 'CapEx': c.get('capex')}
            missing_comps_list = [name for name, val in required_comps.items() if not np.isfinite(val)]
            if not missing_comps_list:
                fcff_val = c['ebit']*(1-tax_rt) + c['dep_amort'] - c['wc'] - c['capex']
        elif method_choice == 1: # NOPAT Approach (Same calculation as 0)
            required_comps = {'EBIT': c.get('ebit'), 'D&A': c.get('dep_amort'), 'WC': c.get('wc'), 'CapEx': c.get('capex')}
            missing_comps_list = [name for name, val in required_comps.items() if not np.isfinite(val)]
            if not missing_comps_list:
                fcff_val = c['ebit']*(1-tax_rt) + c['dep_amort'] - c['wc'] - c['capex'] # Same formula
        elif method_choice == 2: # Cash Flow Statement Adjustment Method (CFO-Based)
            required_comps = {'CFO': c.get('cfo'), 'Interest Exp': c.get('int_exp'), 'CapEx': c.get('capex')}
            missing_comps_list = [name for name, val in required_comps.items() if not np.isfinite(val)]
            if not missing_comps_list:
                fcff_val = c['cfo'] + c['int_exp']*(1-tax_rt) - c['capex']
        elif method_choice == 3: # Bottom-Up Approach (Net Income-Based)
            required_comps = {'Net Income': c.get('net_income'), 'Interest Exp': c.get('int_exp'), 'D&A': c.get('dep_amort'), 'WC': c.get('wc'), 'CapEx': c.get('capex')}
            missing_comps_list = [name for name, val in required_comps.items() if not np.isfinite(val)]
            if not missing_comps_list:
                fcff_val = c['net_income'] + c['int_exp']*(1-tax_rt) + c['dep_amort'] - c['wc'] - c['capex']
        elif method_choice == 4: # EBITDA Proxy Method
            required_comps = {'EBITDA': c.get('ebitda'), 'Tax Provision': c.get('tax_prov'), 'WC': c.get('wc'), 'CapEx': c.get('capex')}
            missing_comps_list = [name for name, val in required_comps.items() if not np.isfinite(val)]
            if not missing_comps_list:
                fcff_val = c['ebitda'] - c['tax_prov'] - c['wc'] - c['capex']
        else:
            log_list_parent.append(f"    ❌ Invalid method choice: {method_choice}.")
            return np.nan

        # Log missing components if any
        if missing_comps_list:
            log_list_parent.append(f"    ❌ Missing Component(s) for Method {method_choice}: {', '.join(missing_comps_list)}")
            return np.nan

        # Check final calculated value
        if np.isfinite(fcff_val):
            log_list_parent.append(f"    Calc FCFF = {format_currency(fcff_val)}")
        else:
            log_list_parent.append(f"    Calculation resulted in Non-Finite value.")
            fcff_val = np.nan # Ensure non-finite result is NaN

    except Exception as e:
        log_list_parent.append(f"    ❌ Calculation Error: {e}")
        print(traceback.format_exc())
        return np.nan

    return fcff_val # Return calculated value or NaN


# --- End Helper Functions ---

# %% [markdown]
# ## 3. User Input & Dynamic Fetching: Ticker, Rf, Rm, Benchmarks, Beta, Spread
# (Section Unchanged)

# %%
input_sources = {}
user_ticker = input(f"Enter ticker [Default: {DEFAULT_TICKER}]: ") or DEFAULT_TICKER
ticker_symbol = user_ticker.strip().upper(); display(Markdown(f"**Ticker:** `{ticker_symbol}`"))

is_indian_stock = ticker_symbol.endswith((".NS", ".BO"))
if is_indian_stock: print("Info: Indian stock detected (.NS or .BO suffix).")
# Suggest appropriate Rf proxy based on market
rf_proxy_ticker = "^NS10" if is_indian_stock else "^TNX"; print(f"Info: Using suggested Rf proxy for dynamic fetch: {rf_proxy_ticker}")

print("\n--- CAPM Inputs ---")
dynamic_rf_rate = try_get_dynamic_rf(rf_proxy_ticker) # Returns rate (e.g., 0.07) or None

# Use dynamic Rf if available and valid, otherwise use default suggestion
suggested_rf_val = dynamic_rf_rate * 100 if dynamic_rf_rate else DEFAULT_RF_RATE_SUGGESTION
rf_prompt = f"Enter Risk-Free Rate (Rf) %"
# Get input as percentage value (e.g., user enters '7' for 7%)
risk_free_rate_input_pct = get_validated_float_input(rf_prompt, default_value=f"{suggested_rf_val:.2f}", min_val=0.0, max_val=20.0, is_percentage=True)
risk_free_rate_input = risk_free_rate_input_pct / 100.0 # Convert to rate (e.g., 0.07)
input_sources['Rf'] = 'Dynamic Fetch' if dynamic_rf_rate and abs(risk_free_rate_input - dynamic_rf_rate) < 1e-4 else 'User Input/Default'
display(Markdown(f"**Rf:** {format_percent(risk_free_rate_input)} (`Source: {input_sources['Rf']}`)"))

# Market Return (Rm) - suggest based on Rf + default MRP
suggested_mrp = DEFAULT_MARKET_RISK_PREMIUM
suggested_rm = risk_free_rate_input + suggested_mrp
rm_prompt = f"Enter Expected Market Return (Rm) %. Suggestion based on Rf + {format_percent(suggested_mrp)} MRP: {format_percent(suggested_rm)}"
while True:
    market_return_input_pct = get_validated_float_input(rm_prompt, default_value=f"{suggested_rm*100:.1f}", min_val=-10.0, max_val=35.0, is_percentage=True)
    market_return_input = market_return_input_pct / 100.0 # Convert to rate
    if market_return_input >= risk_free_rate_input:
        break
    else: print(f"❌ Market Return (Rm) {format_percent(market_return_input)} must be greater than or equal to Risk-Free Rate (Rf) {format_percent(risk_free_rate_input)}. Please re-enter.")

market_risk_premium_input = market_return_input - risk_free_rate_input
input_sources['Rm'] = 'User Input/Default'
input_sources['MRP'] = 'Derived (Rm - Rf)'
display(Markdown(f"**Rm:** {format_percent(market_return_input)} (`Source: {input_sources['Rm']}`)"))
display(Markdown(f"**MRP:** {format_percent(market_risk_premium_input)} (`Source: {input_sources['MRP']}`)"))

# Initialize Beta and Credit Spread (will be fetched/asked for later if needed)
beta_input = None; input_sources['Beta'] = 'Pending Fetch/Input'
user_credit_spread = None; input_sources['Credit Spread'] = 'Pending WACC Calc'

print("\n--- Ratio Benchmarks ---"); display(Markdown("Enter benchmark values for comparison, or leave blank to use defaults."))
user_benchmarks={}; benchmark_sources={}; print("Provide benchmark ratios:")
for ratio_key in REQUIRED_BENCHMARK_RATIOS:
    default_val = DEFAULT_BENCHMARK_RATIOS.get(ratio_key)
    prompt = f"  Benchmark {ratio_key}"
    # Determine if the ratio is typically expressed as a percentage
    is_perc = ratio_key in ['ROE','ROA','ROCE']
    # Set reasonable min/max bounds for input validation
    min_b, max_b = (-100, 200) if is_perc else (-100, 500) # Wider range for non-percentage ratios

    # Get validated input (allow blank to use default)
    # Note: get_validated_float_input returns the number as entered (e.g., 15 for 15%)
    bench_val_input = get_validated_float_input(prompt, default_value=f"{default_val:.2f}" if default_val is not None else None, allow_blank=True, min_val=min_b, max_val=max_b, is_percentage=is_perc)

    final_bench_val = np.nan # Initialize as NaN
    source = 'N/A'

    if bench_val_input is not None: # User provided a value
        # Convert to rate if it was a percentage input
        final_bench_val = bench_val_input / 100.0 if is_perc else bench_val_input
        user_benchmarks[ratio_key] = final_bench_val
        benchmark_sources[ratio_key] = 'User Input'
        source = 'User Input'
    elif default_val is not None: # User left blank, use default if available
        # Defaults are already stored as rates (e.g., ROE=19.05 means 0.1905)
        final_bench_val = default_val / 100.0 if is_perc else default_val # Ensure default is also treated as rate if perc
        user_benchmarks[ratio_key] = final_bench_val
        benchmark_sources[ratio_key] = 'Default'
        source = 'Default'
    else: # User left blank, no default available
        user_benchmarks[ratio_key] = np.nan
        benchmark_sources[ratio_key] = 'N/A'
        source = 'N/A'

    # Display the final benchmark value being used
    disp_val = "N/A"
    if np.isfinite(final_bench_val):
        # Use format_percent for ROE/ROA/ROCE, otherwise format as number
        disp_val = format_percent(final_bench_val) if is_perc else f"{final_bench_val:.2f}"

    display(Markdown(f"  - **{ratio_key}:** {disp_val} (`Source: {source}`)"))

input_sources['Benchmarks'] = 'User/Default Mix'


# %% [markdown]
# ## 4. Data Acquisition (Fetching Extended History)
# (Section Unchanged)

# %%
stock_data = {}
try:
    stock = yf.Ticker(ticker_symbol)
    # Attempt to fetch basic info first
    info = stock.info
    # Check if info is valid and contains a price; if not, try history as a fallback check
    if not info or not info.get('regularMarketPrice'):
        print(f"Warning: Basic info dictionary seems limited or missing price for {ticker_symbol}. Attempting to fetch history to confirm ticker validity...")
        hist_check = stock.history(period="1mo") # Fetch a short period just to see if ticker works
        if hist_check.empty:
            raise ValueError(f"Ticker '{ticker_symbol}' might be invalid or delisted. No basic info or recent history found.")
        else:
             print(f"Info: History data found for {ticker_symbol}. Proceeding, but some info fields might be missing.")
             info = {} if not isinstance(info, dict) else info # Ensure info is a dict, even if empty

    stock_data['info'] = info

    # Fetch financial statements (annual first, fallback to quarterly)
    stock_data['financials'] = stock.financials
    stock_data['balance_sheet'] = stock.balance_sheet
    stock_data['cashflow'] = stock.cashflow

    if stock_data['financials'].empty: print("Warning: Annual Income Statement empty, trying quarterly financials..."); stock_data['financials']=stock.quarterly_financials
    if stock_data['balance_sheet'].empty: print("Warning: Annual Balance Sheet empty, trying quarterly balance sheet..."); stock_data['balance_sheet']=stock.quarterly_balance_sheet
    if stock_data['cashflow'].empty: print("Warning: Annual Cash Flow statement empty, trying quarterly cashflow..."); stock_data['cashflow']=stock.quarterly_cashflow

    # Check again after fetching quarterly data
    missing_statements = [s for s in ['financials','balance_sheet','cashflow'] if stock_data.get(s, pd.DataFrame()).empty]
    if missing_statements:
        print(f"⚠️ CRITICAL WARNING: Missing essential financial statement(s): {', '.join(missing_statements)}. DCF and Ratio analysis will likely fail or be highly unreliable.")

    # Fetch historical price data
    stock_data['history'] = stock.history(period="2y") # For recent trends, maybe beta calc if needed
    stock_data['history_long'] = stock.history(period="4y", interval="1d") # For growth calcs / context

    if stock_data['history_long'] is not None and not stock_data['history_long'].empty:
        print(f"Info: Fetched long price history ({len(stock_data['history_long'])} days over approx 4 years).")
        if len(stock_data['history_long']) < 2: print(f"⚠️ Warning: Long history has fewer than 2 data points.")
    else: print(f"⚠️ Warning: Could not fetch long price history (4y).")

    # Extract key info, with fallbacks
    company_name = info.get("longName", info.get("shortName", ticker_symbol)) # Use shortName if longName missing
    currency_code = info.get("currency", DEFAULT_CURRENCY_SYMBOL)
    if not isinstance(currency_code, str) or len(currency_code) > 5: currency_code = DEFAULT_CURRENCY_SYMBOL # Sanitize currency
    currency_symbol = "₹" # Forcing Rupee symbol due to V21.6 formatting requirement

    # Get current price with fallbacks
    price_keys = ['currentPrice', 'regularMarketPrice', 'previousClose', 'regularMarketPreviousClose']
    current_price = safe_get(info, price_keys, 'Info', default=np.nan)
    price_source = "Info (current/regular/previous)" if np.isfinite(current_price) else "N/A from Info"

    # Fallback to history if info price is missing
    if not np.isfinite(current_price) and stock_data['history'] is not None and not stock_data['history'].empty:
        if 'Close' in stock_data['history'].columns and not stock_data['history']['Close'].empty:
            last_close = stock_data['history']['Close'].iloc[-1]
            if pd.notna(last_close) and last_close > 0:
                current_price = last_close
                price_source = "History (Last Close)"
                print(f"Info: Used last closing price from history as current price.")
            else: price_source += ", History Invalid"
        else: price_source += ", History Missing Close"

    if not np.isfinite(current_price):
        print(f"⚠️ WARNING: Could not determine current price for {ticker_symbol}.")
        current_price = np.nan # Ensure it's NaN if not found
        price_source += ", History Not Checked or Invalid" # Update source if price is still NaN

    # Get Shares Outstanding with fallbacks
    shares_outstanding = safe_get(info, ['sharesOutstanding'], 'Info', default=0)
    shares_source = "Info"
    if not np.isfinite(shares_outstanding) or shares_outstanding <= 0:
        print("Warning: Shares outstanding invalid or missing from info. Trying Balance Sheet...")
        shares_source = "Info Invalid"
        if stock_data['balance_sheet'] is not None and not stock_data['balance_sheet'].empty:
            bs_latest = stock_data['balance_sheet'].iloc[:, 0]
            shares_bs_keys = ['Share Issued', 'Ordinary Shares Number', 'Total Common Shares Outstanding']
            shares_bs = safe_get(bs_latest, shares_bs_keys, 'Balance Sheet', 0)
            if shares_bs is not None and np.isfinite(shares_bs) and shares_bs > 0:
                shares_outstanding = shares_bs
                shares_source = "Balance Sheet"
                print(f"Info: Using shares outstanding from Balance Sheet: {shares_outstanding:,.0f}")
            else:
                print("Warning: Could not find valid shares outstanding on the Balance Sheet.")
                shares_source += ", BS Invalid/Missing"
                shares_outstanding = 0
        else:
            print("Warning: Cannot check Balance Sheet for shares (BS missing or empty).")
            shares_source += ", BS Unavailable"
            shares_outstanding = 0
        shares_outstanding = float(shares_outstanding) if np.isfinite(shares_outstanding) else 0.0

    # Get Beta - Try fetching dynamically, prompt user if fails
    beta_input = safe_get(info, ['beta'], 'Info', default=None)
    if beta_input is not None and np.isfinite(beta_input):
        input_sources['Beta'] = 'Dynamic Fetch'
        display(Markdown(f"**Beta (β):** {beta_input:.2f} (`Source: {input_sources['Beta']}`)"))
    else:
        print(f"⚠️ WARNING: Beta not available or invalid from dynamic fetch ({beta_input}).")
        beta_user = get_validated_float_input(f"Enter Beta (β) for {ticker_symbol} (leave blank for 1.0):", default_value="1.0", allow_blank=True, min_val=-1.0, max_val=4.0)
        beta_input = float(beta_user) if beta_user is not None else 1.0
        info['beta'] = beta_input
        input_sources['Beta'] = 'User Input' if beta_user is not None else 'Default (1.0)'
        display(Markdown(f"**Beta (β):** {beta_input:.2f} (`Source: {input_sources['Beta']}`)"))

    # Market Cap - Try fetching, fallback to calculation
    market_cap = safe_get(info, ['marketCap'], 'Info', default=0)
    mcap_source = "Info"
    if (not np.isfinite(market_cap) or market_cap <= 0):
         mcap_source = "Info Invalid/Missing"
         if shares_outstanding > 0 and np.isfinite(current_price) and current_price > 0:
            market_cap = shares_outstanding * current_price
            info['marketCap'] = market_cap
            mcap_source = "Calculated (Price * Shares)"
            print(f"Info: Market Cap calculated from price and shares.")
         else:
            print(f"⚠️ WARNING: Market Cap N/A (cannot fetch or calculate).")
            market_cap = np.nan
            mcap_source = "N/A (Fetch & Calc Failed)"
    elif market_cap <=0:
         market_cap = np.nan
         mcap_source = "Info Zero/Invalid"
         print(f"⚠️ WARNING: Market Cap from info is zero or invalid.")

    # Other Info
    sector = info.get("sector", "N/A"); industry = info.get("industry", "N/A"); summary = info.get("longBusinessSummary", "N/A.")

    # --- Display Summary ---
    display(Markdown(f"--- **Company Overview** ---"))
    display(Markdown(f"**Company:** {company_name}"))
    display(Markdown(f"**Sector:** {sector} | **Industry:** {industry}"))
    display(Markdown(f"**Currency:** {currency_code} (Using Symbol: {currency_symbol})"))
    display(Markdown(f"**Current Price:** {format_currency(current_price)} (`Source: {price_source}`)"))
    display(Markdown(f"**Market Cap:** {format_currency(market_cap)} (`Source: {mcap_source}`)"))
    display(Markdown(f"**Shares Outstanding:** {shares_outstanding:,.0f} (`Source: {shares_source}`)"))
    if len(summary) > 5 and summary != "N/A.": display(Markdown(f"**Business Summary:** {summary[:300]}..."))

except Exception as e:
    print(f"❌ CRITICAL ERROR during data acquisition for {ticker_symbol}: {e}")
    print(traceback.format_exc())
    sys.exit("Halting execution due to data fetch error.")

# --- Final Sanity Checks Before Proceeding ---
if not isinstance(shares_outstanding,(int,float)) or shares_outstanding <= 0:
    print("CRITICAL: Shares outstanding is still invalid after fetch and fallbacks.")
    shares_outstanding = get_validated_float_input(f"CRITICAL: Shares invalid ({shares_outstanding}). Enter Shares Outstanding:", min_val=1)
    shares_outstanding = float(shares_outstanding)
    input_sources['Shares'] = 'User Input (Critical Override)'
    print(f"Using user-provided shares: {shares_outstanding:,.0f}")

if beta_input is None or not np.isfinite(beta_input):
    print("CRITICAL: Beta is invalid after fetch and user prompt. Defaulting to 1.0.")
    beta_input = 1.0
    input_sources['Beta'] = 'Default (1.0) [Critical Override]'

if not np.isfinite(market_cap):
    print(f"⚠️ WARNING: Market Cap remains invalid. EV/EBITDA and WACC weights might be affected.")
    market_cap = np.nan

if not np.isfinite(current_price):
    print(f"⚠️ WARNING: Current Price remains invalid. P/E, P/B ratios and comparison will be affected.")
    current_price = np.nan


# %% [markdown]
# ## 5. WACC Calculation
# (Section Unchanged)

# %%
# WACC Function Definition
def calculate_wacc(stock_info_dict, financials_df, balance_sheet_df, user_rf, user_mrp, user_beta, input_sources_dict, currency_sym):
    global user_credit_spread # Needed to potentially *set* the global var if prompted inside

    display(Markdown("### Calculating Weighted Average Cost of Capital (WACC)..."))
    components = {} # To store parts of the WACC calculation
    calculation_log = [] # To store step-by-step process for display
    calculation_log.append(f"Input Sources -> Rf: {input_sources_dict.get('Rf','?')}, MRP: {input_sources_dict.get('MRP','?')}, Beta: {input_sources_dict.get('Beta','?')}")

    # --- Cost of Equity (Ke) ---
    beta_val = user_beta
    if beta_val <= 0: calculation_log.append(f"⚠️ Warning: Beta is non-positive ({beta_val:.2f}).")
    components['beta'] = beta_val
    components['risk_free_rate'] = user_rf
    components['market_risk_premium'] = user_mrp
    cost_of_equity = user_rf + beta_val * user_mrp
    if not (0 < cost_of_equity < 0.50): calculation_log.append(f"⚠️ Warning: Calculated Cost of Equity ({format_percent(cost_of_equity)}) seems unusual.")
    components['cost_of_equity'] = cost_of_equity
    calculation_log.append(f"Inputs -> Rf: {format_percent(user_rf)}, Beta: {beta_val:.2f}, MRP: {format_percent(user_mrp)}")
    calculation_log.append(f"↳ Cost of Equity (Ke = Rf + Beta * MRP): {format_percent(cost_of_equity)}")

    # --- Cost of Debt (Kd) & Tax Rate ---
    cost_of_debt_pre_tax = np.nan; tax_rate = np.nan; after_tax_cost_of_debt = np.nan
    total_debt = 0.0; debt_calculation_method = "N/A"; tax_rate_source = "N/A"
    statements_available = (financials_df is not None and not financials_df.empty and
                            balance_sheet_df is not None and not balance_sheet_df.empty)

    if statements_available:
        latest_bs = balance_sheet_df.iloc[:, 0]; latest_fin = financials_df.iloc[:, 0]
        debt_keys_long = ['Total Debt Non Current','Long Term Debt', 'Total Non Current Liabilities Net Minority Interest']; debt_keys_short = ['Total Debt Current','Current Debt', 'Total Current Liabilities']
        long_term_debt = safe_get(latest_bs, debt_keys_long, 'BS(Long Debt)', 0); short_term_debt = safe_get(latest_bs, debt_keys_short, 'BS(Short Debt)', 0)
        long_term_debt = float(long_term_debt) if np.isfinite(long_term_debt) else 0.0; short_term_debt = float(short_term_debt) if np.isfinite(short_term_debt) else 0.0
        total_debt = long_term_debt + short_term_debt
        components['debt_value_used'] = total_debt
        calculation_log.append(f"Debt from BS -> LT Debt: {format_currency(long_term_debt)}, ST Debt: {format_currency(short_term_debt)} -> Total Debt: {format_currency(total_debt)}")
        interest_keys = ['Interest Expense','Interest Expense Non Operating', 'Interest Expense Net']
        interest_expense = abs(safe_get(latest_fin, interest_keys, 'IS(Interest)', 0))
        interest_expense = float(interest_expense) if np.isfinite(interest_expense) else 0.0
        calculation_log.append(f"Interest from IS -> Interest Expense: {format_currency(interest_expense)}")
        implied_rate_calculated = False
        if total_debt > 1000 and interest_expense > 0:
             implied_rate = interest_expense / total_debt; implied_rate_calculated = True
             calculation_log.append(f"Implied Pre-tax Kd (Interest / Debt) = {format_percent(implied_rate)}")
             if MIN_COST_OF_DEBT <= implied_rate <= MAX_COST_OF_DEBT:
                 cost_of_debt_pre_tax = implied_rate; debt_calculation_method = "Implied (Interest/Debt)"
                 calculation_log.append(f"↳ Using Implied Kd: {format_percent(cost_of_debt_pre_tax)}")
             else:
                 calculation_log.append(f"⚠️ Implied Kd out of bounds. Will use Rf + Spread method."); implied_rate_calculated = False
        elif total_debt > 1000: calculation_log.append(f"ℹ️ Cannot use implied Kd method (Interest Expense is zero). Will use Rf + Spread method.")
        else: calculation_log.append(f"ℹ️ Cannot use implied Kd method (Total Debt is zero or negligible).")

        if not implied_rate_calculated:
             if total_debt > 1000:
                 debt_calculation_method = "Default (Rf + Spread)"
                 if user_credit_spread is None:
                     spread_prompt = f"Implied Kd calculation failed or not applicable. Enter Credit Spread % for {ticker_symbol} over Rf ({format_percent(user_rf)})"
                     user_credit_spread_pct = get_validated_float_input(spread_prompt, default_value=f"{DEFAULT_CREDIT_SPREAD*100:.1f}", allow_blank=False, min_val=0.1, max_val=15.0, is_percentage=True)
                     user_credit_spread = user_credit_spread_pct / 100.0
                     input_sources_dict['Credit Spread'] = 'User Input (for Default Kd)'
                 else: input_sources_dict['Credit Spread'] = 'User Input (Reused)'
                 cost_of_debt_pre_tax = user_rf + user_credit_spread; original_kd = cost_of_debt_pre_tax
                 cost_of_debt_pre_tax = min(max(cost_of_debt_pre_tax, MIN_COST_OF_DEBT), MAX_COST_OF_DEBT)
                 calculation_log.append(f"Default Pre-tax Kd = Rf + Spread = {format_percent(user_rf)} + {format_percent(user_credit_spread)} = {format_percent(original_kd)}")
                 if abs(cost_of_debt_pre_tax - original_kd) > 1e-6: calculation_log.append(f"↳ Bounded Default Kd: {format_percent(cost_of_debt_pre_tax)}")
                 else: calculation_log.append(f"↳ Using Default Kd: {format_percent(cost_of_debt_pre_tax)}")
                 calculation_log.append(f"   (Credit Spread Source: {input_sources_dict['Credit Spread']})")
             else:
                 debt_calculation_method = "Zero Debt"; cost_of_debt_pre_tax = 0.0
                 input_sources_dict['Credit Spread'] = 'N/A (Zero Debt)'; calculation_log.append(f"ℹ️ Pre-tax Kd set to 0% (Zero Debt).")

        tax_exp_keys = ['Tax Provision','Income Tax Expense']; pretax_inc_keys = ['Pretax Income','EBT', 'Earnings Before Tax']
        income_tax_expense = safe_get(latest_fin, tax_exp_keys, 'IS(Tax Exp)', 0); pretax_income = safe_get(latest_fin, pretax_inc_keys, 'IS(Pretax Inc)', 0)
        income_tax_expense = float(income_tax_expense) if np.isfinite(income_tax_expense) else 0.0; pretax_income = float(pretax_income) if np.isfinite(pretax_income) else 0.0
        calculation_log.append(f"Tax Rate Calc -> Tax Expense: {format_currency(income_tax_expense)}, Pre-tax Income: {format_currency(pretax_income)}")
        if pretax_income <= 1 and income_tax_expense > 0:
             ebit_val = safe_get(latest_fin, ["EBIT"], 'IS(EBIT for Tax Proxy)', default=0); ebit_val = float(ebit_val) if np.isfinite(ebit_val) else 0.0
             if ebit_val > 1 and np.isfinite(interest_expense):
                 pretax_proxy = ebit_val - interest_expense
                 if pretax_proxy > 1: pretax_income = pretax_proxy; calculation_log.append(f"ℹ️ Using EBIT - Interest proxy for Pretax Income: {format_currency(pretax_income)}")
                 else: calculation_log.append(f"⚠️ Cannot calc tax rate (Pretax & Proxy too low).")
             else: calculation_log.append(f"⚠️ Cannot calc tax rate (Pretax low, EBIT low/NA).")

        if pretax_income > 1 and income_tax_expense >= 0:
            calc_tax_rate = income_tax_expense / pretax_income; calculation_log.append(f"Calculated Effective Tax Rate = {format_percent(calc_tax_rate)}")
            original_tax_rate = calc_tax_rate; tax_rate = min(max(calc_tax_rate, MIN_TAX_RATE), MAX_TAX_RATE); tax_rate_source = "Derived (Tax Exp / Pretax Inc)"
            if abs(tax_rate - original_tax_rate) > 0.001: tax_rate_source += f" [Bounded]"; calculation_log.append(f"↳ Bounded Effective Tax Rate: {format_percent(tax_rate)}")
            else: calculation_log.append(f"↳ Using Effective Tax Rate: {format_percent(tax_rate)}")
        else:
            tax_rate = DEFAULT_TAX_RATE; tax_rate_source = f"Default ({format_percent(DEFAULT_TAX_RATE)}) [Derived Failed]"; calculation_log.append(f"⚠️ Could not derive tax rate. Using default: {format_percent(tax_rate)}")
    else:
        calculation_log.append("⚠️ Statements missing. Using defaults for Kd & Tax."); debt_calculation_method = "Default (No Fin.)"; tax_rate_source = f"Default ({format_percent(DEFAULT_TAX_RATE)}) [No Fin.]"
        if user_credit_spread is None:
             spread_prompt=f"Financials missing. Enter Credit Spread %"; user_credit_spread_pct = get_validated_float_input(spread_prompt, default_value=f"{DEFAULT_CREDIT_SPREAD*100:.1f}", min_val=0.1, max_val=15.0, is_percentage=True)
             user_credit_spread = user_credit_spread_pct / 100.0; input_sources_dict['Credit Spread'] = 'User Input (No Fin.)'
        else: input_sources_dict['Credit Spread'] = 'User Input (Reused)'
        cost_of_debt_pre_tax = user_rf + user_credit_spread; original_kd = cost_of_debt_pre_tax; cost_of_debt_pre_tax = min(max(cost_of_debt_pre_tax, MIN_COST_OF_DEBT), MAX_COST_OF_DEBT); tax_rate = DEFAULT_TAX_RATE; total_debt = 0; components['debt_value_used'] = 0
        calculation_log.append(f"Default Pre-tax Kd = Rf + Spread = {format_percent(original_kd)}")
        if abs(cost_of_debt_pre_tax - original_kd) > 1e-6: calculation_log.append(f"↳ Bounded Default Kd: {format_percent(cost_of_debt_pre_tax)}")
        else: calculation_log.append(f"↳ Using Default Kd: {format_percent(cost_of_debt_pre_tax)}")
        calculation_log.append(f"Using Default Tax Rate: {format_percent(tax_rate)}")

    components['cost_of_debt'] = cost_of_debt_pre_tax if np.isfinite(cost_of_debt_pre_tax) else 0.0
    components['debt_calc_method'] = debt_calculation_method
    components['tax_rate'] = tax_rate if np.isfinite(tax_rate) else DEFAULT_TAX_RATE
    components['tax_rate_source'] = tax_rate_source
    if np.isfinite(components['cost_of_debt']) and np.isfinite(components['tax_rate']): after_tax_cost_of_debt = components['cost_of_debt'] * (1 - components['tax_rate'])
    else: calculation_log.append(f"⚠️ Cannot calculate after-tax Kd."); after_tax_cost_of_debt = np.nan
    components['after_tax_cost_of_debt'] = after_tax_cost_of_debt
    if np.isfinite(after_tax_cost_of_debt): calculation_log.append(f"↳ Cost of Debt (After-tax Kd*(1-T)): {format_percent(after_tax_cost_of_debt)}")

    # --- Market Value Weights (Equity & Debt) ---
    market_cap_val = safe_get(stock_info_dict, ['marketCap'], 'Info', default=np.nan)
    equity_value_for_wacc = np.nan; equity_source = "N/A"
    if np.isfinite(market_cap_val) and market_cap_val > 0: equity_value_for_wacc = market_cap_val; equity_source = "Market Cap"
    else:
        calculation_log.append(f"⚠️ Market Cap invalid. Fallback: Book Equity.")
        equity_source = "Market Cap Invalid"
        if balance_sheet_df is not None and not balance_sheet_df.empty:
            equity_keys = ['Total Stockholder Equity', 'Common Stock Equity', 'Stockholders Equity']
            book_equity_val = safe_get(balance_sheet_df.iloc[:, 0], equity_keys, 'BS(Book Eq)', default=np.nan)
            if np.isfinite(book_equity_val) and book_equity_val > 1.0: equity_value_for_wacc = book_equity_val; equity_source = "Book Equity Fallback"; calculation_log.append(f"↳ Using Book Equity: {format_currency(equity_value_for_wacc)}")
            else: equity_source += ", Book Invalid"; calculation_log.append(f"⚠️ Book Equity invalid.")
        else: equity_source += ", No BS"; calculation_log.append(f"⚠️ Cannot use Book Equity (No BS).")
        if not np.isfinite(equity_value_for_wacc): equity_value_for_wacc = 1.0; equity_source = "Placeholder (1.0)"; calculation_log.append("⚠️ CRITICAL: Using placeholder Equity (1.0).")
    components['equity_value_used'] = equity_value_for_wacc; components['equity_source'] = equity_source
    equity_weight = np.nan; debt_weight = np.nan; weights_source = "N/A"; total_capital = equity_value_for_wacc + total_debt
    if np.isfinite(total_capital) and total_capital > 1.0:
        equity_weight = equity_value_for_wacc / total_capital; debt_weight = total_debt / total_capital; weights_source = "Calculated"
        total_weight = equity_weight + debt_weight
        if abs(total_weight - 1.0) > 1e-6: calculation_log.append(f"ℹ️ Normalizing weights."); equity_weight /= total_weight; debt_weight /= total_weight
    else: calculation_log.append(f"❌ ERROR: Total Cap invalid. Default weights (70/30)."); equity_weight = 0.7; debt_weight = 0.3; weights_source = "Default (Invalid Cap)"
    components['equity_weight'] = equity_weight; components['debt_weight'] = debt_weight; components['weights_source'] = weights_source
    calculation_log.append(f"Capital Values -> Equity ({components['equity_source']}): {format_currency(components['equity_value_used'])}, Debt ({components['debt_calc_method']}): {format_currency(total_debt)}")
    calculation_log.append(f"↳ Weights -> E/V: {format_percent(equity_weight)}, D/V: {format_percent(debt_weight)} ({weights_source})")

    # --- Final WACC Calculation ---
    wacc = np.nan
    if all(map(np.isfinite, [equity_weight, cost_of_equity, debt_weight, after_tax_cost_of_debt])):
        wacc = (equity_weight * cost_of_equity) + (debt_weight * after_tax_cost_of_debt); original_wacc = wacc
        wacc = min(max(wacc, MIN_WACC), MAX_WACC); bound_note = ""
        if abs(wacc - original_wacc) > 0.0001 : bound_note = f" (Bounded)"; calculation_log.append(f"ℹ️ WACC bounded.")
        calculation_log.append(f"▶️ WACC = (E/V * Ke) + (D/V * Kd * (1-T)) = {format_percent(wacc)}{bound_note}")
    else: calculation_log.append(f"❌ ERROR: Cannot calculate final WACC (Components invalid)."); wacc = np.nan
    components['final_wacc'] = wacc; display(Markdown("```\n"+"\n".join(calculation_log)+"\n```"))

    # --- WACC Composition Pie Chart ---
    if np.isfinite(wacc) and wacc > 0 and np.isfinite(equity_weight) and np.isfinite(debt_weight):
        equity_contrib = equity_weight * cost_of_equity; debt_contrib = debt_weight * after_tax_cost_of_debt
        if np.isfinite(equity_contrib) and np.isfinite(debt_contrib) and abs(equity_contrib + debt_contrib - wacc) < 0.01:
           labels = [f'Equity Contribution ({format_percent(equity_contrib)})', f'Debt Contribution ({format_percent(debt_contrib)})']
           values = [equity_contrib, debt_contrib]; fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4, marker_colors=px.colors.qualitative.Pastel, textinfo='percent', sort=False, hoverinfo='label+value')])
           fig.update_layout(title_text=f'WACC Composition (Total: {format_percent(wacc)})', title_x=0.5, showlegend=True, legend_title_text='Components', height=350, margin=dict(t=60, b=30, l=30, r=30)); fig.show()
        else: display(Markdown(f"*(WACC contribution invalid. No chart.)*"))
    elif not np.isfinite(wacc): display(Markdown(f"*(WACC failed. No chart)*"))
    else: display(Markdown(f"*(WACC 0/neg. No chart)*"))
    return wacc, components

# --- Execute WACC Calculation ---
discount_rate, wacc_components = calculate_wacc(stock_data['info'], stock_data.get('financials'), stock_data.get('balance_sheet'), risk_free_rate_input, market_risk_premium_input, beta_input, input_sources, currency_symbol)


# %% [markdown]
# ## 6. FCFF Method Selection (Updated V21.7 - 5 Methods)

# %%
# --- Pre-calculate Averages and Get User Choice ---
selected_fcff_method = None
precalculated_averages = {}
fcff_choice_log = []

print("\n--- FCFF Calculation Method Selection ---")
display(Markdown(("*Note: Method 6 (Unlevered NI) was omitted as it's mathematically identical to Method 3 (Net Income-Based). "
                 "Method 7 (Forecast-Based) cannot be used for historical averaging.*")))
fcff_choice_log.append("--- Pre-calculating 4yr Average FCFF for Each Method (0-4) ---")

# Use global currency symbol determined earlier ('₹')
global_currency_symbol = currency_symbol

# Get the tax rate calculated during WACC, with fallback
wacc_tax_rate = wacc_components.get('tax_rate', DEFAULT_TAX_RATE) if isinstance(wacc_components, dict) else DEFAULT_TAX_RATE
if not np.isfinite(wacc_tax_rate):
    fcff_choice_log.append(f"  ⚠️ Warning: Invalid tax rate from WACC calculation ({wacc_tax_rate}). Using default {format_percent(DEFAULT_TAX_RATE)} for FCFF pre-calculation.")
    wacc_tax_rate = DEFAULT_TAX_RATE
fcff_choice_log.append(f"  Using Tax Rate for pre-calc: {format_percent(wacc_tax_rate)} (Source: {wacc_components.get('tax_rate_source','Default Fallback')})")

# Check if all necessary dataframes are available and tax rate is valid
data_ok_for_fcff = (stock_data.get('financials') is not None and not stock_data['financials'].empty and
                    stock_data.get('balance_sheet') is not None and not stock_data['balance_sheet'].empty and
                    stock_data.get('cashflow') is not None and not stock_data['cashflow'].empty and
                    np.isfinite(wacc_tax_rate))

if data_ok_for_fcff:
    num_hist_years_fcff = 4 # Target number of years for averaging
    num_statement_years_fcff = 0
    try: num_statement_years_fcff = min(stock_data['financials'].shape[1], stock_data['balance_sheet'].shape[1], stock_data['cashflow'].shape[1])
    except Exception as e: fcff_choice_log.append(f"  ❌ ERROR determining available statement years: {e}")

    years_to_avg_fcff = min(num_statement_years_fcff, num_hist_years_fcff)

    if years_to_avg_fcff < 1:
        fcff_choice_log.append(f"  ❌ ERROR: Insufficient historical data available ({years_to_avg_fcff} years) to calculate average FCFF.")
        data_ok_for_fcff = False
    else:
        fcff_choice_log.append(f"  Calculating averages over the last {years_to_avg_fcff} available year(s).")
        # Iterate through the NEW method definitions (0-4)
        for method_key, method_info in FCFF_METHODS_DEFINITION.items():
            fcff_choice_log.append(f"\n  Avg for Method {method_key}: {method_info['name']}...")
            temp_hist_fcff = []
            for i in range(years_to_avg_fcff):
                 if i >= stock_data['financials'].shape[1] or \
                    i >= stock_data['balance_sheet'].shape[1] or \
                    i >= stock_data['cashflow'].shape[1]:
                     temp_hist_fcff.append(np.nan); continue
                 fin_c = stock_data['financials'].iloc[:, i]; bs_c = stock_data['balance_sheet'].iloc[:, i]; cf_c = stock_data['cashflow'].iloc[:, i]
                 bs_p = stock_data['balance_sheet'].iloc[:, i+1] if (i+1) < stock_data['balance_sheet'].shape[1] else None
                 temp_calc_log_for_year = []
                 fcff_yr_val = calculate_fcff_single_method_single_year(method_key, fin_c, bs_c, cf_c, bs_p, wacc_tax_rate, temp_calc_log_for_year)
                 temp_hist_fcff.append(fcff_yr_val)

            valid_fcff_values = [f for f in temp_hist_fcff if np.isfinite(f)]; valid_count = len(valid_fcff_values)
            avg_fcff = np.mean(valid_fcff_values) if valid_count > 0 else np.nan
            precalculated_averages[method_key] = avg_fcff
            fcff_choice_log.append(f"    Individual Year FCFFs ({years_to_avg_fcff} yrs): {[format_currency(f) for f in temp_hist_fcff]}")
            if valid_count > 0: fcff_choice_log.append(f"    Resulting {valid_count}-year Avg FCFF: {format_currency(avg_fcff)}")
            else: fcff_choice_log.append(f"    Resulting Avg FCFF: N/A (No valid yearly calculations)")

    # --- Present Choices to User ---
    print("\nPlease choose the FCFF calculation method based on the calculated historical averages:")
    valid_method_choices = []; display_options = []
    # Iterate through the NEW method definitions (0-4)
    for method_key, method_info in FCFF_METHODS_DEFINITION.items():
        avg_val = precalculated_averages.get(method_key, np.nan)
        avg_display = "N/A (Calc Failed/No Data)"; is_positive_and_valid = False
        if np.isfinite(avg_val):
            avg_display = format_currency(avg_val)
            if avg_val > 1e-9: is_positive_and_valid = True; avg_display += " ✅ (Positive Avg - Usable)"
            else: avg_display += " ⚠️ (≤ 0 Avg - Not Usable)"
        elif method_key not in precalculated_averages: avg_display = "N/A (Not Attempted)"
        option_str = f"\n  {method_key}. {method_info['name']}\n"
        option_str += f"     Formula: {method_info['formula']}\n"
        option_str += f"     {years_to_avg_fcff}yr Avg FCFF: {avg_display}"
        display_options.append(option_str)
        if is_positive_and_valid: valid_method_choices.append(method_key)

    print("--- Available Methods and Average FCFF ---")
    for option in display_options: print(option)
    print("------------------------------------------")

    # --- Get User Selection ---
    if not valid_method_choices:
         print("\n❌ CRITICAL: No FCFF methods resulted in a positive average FCFF.")
         print("   Cannot proceed with DCF valuation using the FCFF method.")
         selected_fcff_method = None; input_sources['FCFF Method'] = "None (No Positive Avg FCFF Found)"
    else:
        prompt_text = f"Enter the number ({', '.join(map(str, valid_method_choices))}) for the FCFF method to use:"
        while selected_fcff_method is None:
            try:
                choice_str = input(f"{prompt_text}: ").strip()
                if not choice_str: print("❌ Please enter a valid method number."); continue
                method_idx = int(choice_str)
                if method_idx in valid_method_choices:
                    selected_fcff_method = method_idx
                    method_name_chosen = FCFF_METHODS_DEFINITION[selected_fcff_method]['name']
                    chosen_avg = precalculated_averages[selected_fcff_method]
                    display(Markdown(f"**Selected FCFF Method:** `{method_idx} - {method_name_chosen}`\n"
                                     f"  *Using 4yr Avg Basis: {format_currency(chosen_avg)}*"))
                    input_sources['FCFF Method'] = f"User Choice ({method_name_chosen} - Method {method_idx})"
                else: print(f"❌ Invalid choice. Please enter one of: {valid_method_choices}.")
            except ValueError: print("❌ Invalid input. Please enter a number.")
            except Exception as e: print(f"An unexpected error: {e}"); sys.exit("Halting.")
else:
    print("\n❌ ERROR: Cannot proceed with FCFF method selection (Missing Data or Invalid Tax Rate).")
    fcff_choice_log.append("  ❌ ERROR: Prerequisite data missing."); selected_fcff_method = None
    input_sources['FCFF Method'] = "None (Prerequisite Data Missing)"


# %% [markdown]
# ## 7. FCFF Calculation & Projection (Updated V21.7.1 - TV Syntax Fixed)

# %%
# FCFF Function Definition (V21.7.1 - TV Syntax Fixed)
def calculate_and_project_fcff_user_selected_method_v21_7( # Renamed slightly
                                   user_selected_method_idx, # The method chosen by the user (0-4)
                                   precalculated_avg_basis, # The positive avg FCFF calculated for the chosen method
                                   financials_df, balance_sheet_df, cashflow_df, # Historical data
                                   wacc_comps, # WACC components dict (for tax rate)
                                   disc_rate, # The final WACC discount rate
                                   forecast_yrs, # Number of years to forecast (e.g., 3)
                                   final_user_rf, # User's risk-free rate (for terminal growth cap)
                                   default_init_growth, # Fallback initial growth rate
                                   currency_sym): # Currency symbol ('₹')

    method_info = FCFF_METHODS_DEFINITION.get(user_selected_method_idx, {"name": "Unknown Method"})
    method_name_display = method_info['name']
    display(Markdown(f"### Calculating & Projecting FCFF (Using Selected Method: {user_selected_method_idx} - {method_name_display})"))
    projection_log = []
    projection_results = { # Same structure as before
        'historical_fcffs_selected_method': {}, 'historical_revenues': {},
        'current_fcff_basis': precalculated_avg_basis,
        'fcff_basis_method': f"{method_name_display} (User Selected Method {user_selected_method_idx} - 4yr Avg)",
        'projected_fcffs': [], 'growth_rates': [], 'pv_fcffs': [],
        'cumulative_pv_fcff': np.nan, 'initial_growth_rate': np.nan,
        'growth_source': 'N/A', 'terminal_growth_rate': np.nan,
        'terminal_value': np.nan, 'pv_terminal_value': np.nan, 'error': None
    }

    # --- Pre-checks --- (Same as before)
    if not np.isfinite(disc_rate):
        projection_results['error']="Invalid WACC provided."; projection_log.append(f"❌ ERROR: WACC is not finite ({disc_rate}). Cannot proceed."); display(Markdown("```\n"+"\n".join(projection_log)+"\n```")); return projection_results
    if not np.isfinite(precalculated_avg_basis) or precalculated_avg_basis <= 1e-9:
        projection_results['error']=f"Invalid/non-positive FCFF basis ({format_currency(precalculated_avg_basis)}) for selected method {user_selected_method_idx}. Cannot proceed."; projection_log.append(f"❌ ERROR: {projection_results['error']}"); display(Markdown("```\n"+"\n".join(projection_log)+"\n```")); return projection_results

    effective_tax_rate = wacc_components.get('tax_rate', DEFAULT_TAX_RATE) if isinstance(wacc_components, dict) else DEFAULT_TAX_RATE
    if not np.isfinite(effective_tax_rate): effective_tax_rate = DEFAULT_TAX_RATE
    tax_rate_src = wacc_components.get('tax_rate_source', '?') if isinstance(wacc_components, dict) else 'Default Fallback'
    projection_log.append(f"Using WACC/Discount Rate: {format_percent(disc_rate)}")
    projection_log.append(f"Using Effective Tax Rate: {format_percent(effective_tax_rate)} (Source: {tax_rate_src})")
    projection_log.append(f"FCFF Basis for Projection (T=0): {format_currency(precalculated_avg_basis)} (Source: {projection_results['fcff_basis_method']})")

    # --- PHASE 1: Re-Calculate Historical FCFF & Revenue using the SELECTED method --- (Same logic)
    projection_log.append(f"\n--- Phase 1: Re-Calculating Historical Data (Using ONLY Method {user_selected_method_idx}) ---")
    num_hist_years_needed = 4; num_statement_years_avail = 0
    try: num_statement_years_avail = min(financials_df.shape[1], balance_sheet_df.shape[1], cashflow_df.shape[1])
    except: pass
    hist_fcffs_sel = []; hist_revs = []
    years_to_calc_hist = min(num_statement_years_avail, num_hist_years_needed)
    if years_to_calc_hist < num_hist_years_needed: projection_log.append(f"  ⚠️ Warning: Only {years_to_calc_hist} years of data available for CAGR.")
    for i in range(years_to_calc_hist):
        fin = financials_df.iloc[:, i]; bs = balance_sheet_df.iloc[:, i]; cf = cashflow_df.iloc[:, i]
        bs_p = balance_sheet_df.iloc[:, i+1] if (i+1) < balance_sheet_df.shape[1] else None
        log_for_hist_calc = []
        fcff_val = calculate_fcff_single_method_single_year(user_selected_method_idx, fin, bs, cf, bs_p, effective_tax_rate, log_for_hist_calc)
        hist_fcffs_sel.append(fcff_val)
        rev_val = safe_get(fin, ["Total Revenue", "Revenue"], "IS", default=np.nan)
        hist_revs.append(rev_val)
    projection_log.append(f"  Recalculated Hist FCFF (Method {user_selected_method_idx}, T-0 to T-{years_to_calc_hist-1}): {[format_currency(f) for f in hist_fcffs_sel]}")
    projection_log.append(f"  Corresponding Hist Revenue: {[format_currency(r) for r in hist_revs]}")
    while len(hist_fcffs_sel) < num_hist_years_needed: hist_fcffs_sel.append(np.nan)
    while len(hist_revs) < num_hist_years_needed: hist_revs.append(np.nan)
    projection_results['historical_fcffs_selected_method'] = {f"T-{i}": v for i, v in enumerate(hist_fcffs_sel)}
    projection_results['historical_revenues'] = {f"T-{i}": v for i, v in enumerate(hist_revs)}

    # --- PHASE 2: Estimate Initial Growth Rate (Based on Rev & Selected FCFF CAGR ONLY) --- (Same logic)
    projection_log.append("\n--- Phase 2: Estimating Initial Growth (GM of Revenue & Selected FCFF CAGRs) ---")
    init_growth = np.nan; growth_src = "N/A"; cagr_calc_details = {}; cagr_list_for_gm = []
    num_yrs_cagr = 3
    # Rev CAGR
    rev_cagr = np.nan; projection_log.append(f"\nCalculating Revenue {num_yrs_cagr}yr CAGR...")
    if len(hist_revs) >= num_hist_years_needed:
        rev_end = hist_revs[0]; rev_start = hist_revs[num_yrs_cagr]
        projection_log.append(f"  Revenue Data: Start={format_currency(rev_start)}, End={format_currency(rev_end)}")
        if pd.notna(rev_end) and math.isfinite(rev_end) and pd.notna(rev_start) and math.isfinite(rev_start) and rev_start > 1e-9:
            rev_cagr = calculate_cagr_modified(rev_start, rev_end, num_yrs_cagr)
            if pd.notna(rev_cagr) and math.isfinite(rev_cagr): cagr_list_for_gm.append(rev_cagr); projection_log.append(f"  ✅ Revenue {num_yrs_cagr}yr CAGR: {format_percent(rev_cagr)}")
            else: projection_log.append(f"  ❌ Rev CAGR failed.")
        else: projection_log.append(f"  ❌ Rev CAGR skipped (Invalid data).")
    else: projection_log.append(f"  ❌ Rev CAGR skipped (Insufficient data).")
    # FCFF CAGR (Selected Method)
    fcff_cagr = np.nan; projection_log.append(f"\nCalculating FCFF {num_yrs_cagr}yr CAGR (Method {user_selected_method_idx})...")
    if len(hist_fcffs_sel) >= num_hist_years_needed:
        fcff_end = hist_fcffs_sel[0]; fcff_start = hist_fcffs_sel[num_yrs_cagr]
        projection_log.append(f"  FCFF Data: Start={format_currency(fcff_start)}, End={format_currency(fcff_end)}")
        if pd.notna(fcff_end) and math.isfinite(fcff_end) and fcff_end > 1e-9 and pd.notna(fcff_start) and math.isfinite(fcff_start) and fcff_start > 1e-9:
            fcff_cagr = calculate_cagr_modified(fcff_start, fcff_end, num_yrs_cagr)
            if pd.notna(fcff_cagr) and math.isfinite(fcff_cagr): cagr_list_for_gm.append(fcff_cagr); projection_log.append(f"  ✅ FCFF {num_yrs_cagr}yr CAGR: {format_percent(fcff_cagr)}")
            else: projection_log.append(f"  ❌ FCFF CAGR failed.")
        else: projection_log.append(f"  ❌ FCFF CAGR skipped (Non-positive/invalid data).")
    else: projection_log.append(f"  ❌ FCFF CAGR skipped (Insufficient data).")
    # GM
    projection_log.append("\nCalculating Geometric Mean...")
    if cagr_list_for_gm:
        gm_growth = geometric_mean_modified(cagr_list_for_gm);
        if pd.notna(gm_growth) and math.isfinite(gm_growth): init_growth = gm_growth; growth_src = f"GM of {len(cagr_list_for_gm)} valid CAGR(s)"; projection_log.append(f"  ▶️ Calculated Initial Growth (GM): {format_percent(init_growth)}")
        else: init_growth = default_init_growth; growth_src = f"Default ({format_percent(default_init_growth)}) [GM Fail]"; projection_log.append(f"  ⚠️ GM Failed. Using default.")
    else: init_growth = default_init_growth; growth_src = f"Default ({format_percent(default_init_growth)}) [No CAGRs]"; projection_log.append(f"  ⚠️ No Valid CAGRs. Using Default.")
    # Bounds
    unbounded_growth = init_growth; init_growth = max(init_growth, MIN_INITIAL_GROWTH_FLOOR); init_growth = min(init_growth, MAX_INITIAL_GROWTH_CEILING)
    if abs(init_growth - unbounded_growth) > 1e-6: growth_src += f" (Bounded)"; projection_log.append(f"  Applied Bounds: Final Initial Growth = {format_percent(init_growth)}")
    else: projection_log.append(f"  Final Initial Growth Rate = {format_percent(init_growth)}")
    projection_results['initial_growth_rate'] = init_growth; projection_results['growth_source'] = growth_src

    # --- PHASE 3: Terminal Growth Rate & FCFF Projection --- (TV Block Corrected V21.7.1)
    projection_log.append("\n--- Phase 3: Determining Terminal Growth & Projecting FCFF ---")
    # Determine Terminal Growth Rate (g)
    g_init_final = init_growth if pd.notna(init_growth) and math.isfinite(init_growth) else TERMINAL_GROWTH_RATE_CAP
    g_wacc_cap = disc_rate * 0.60; g_rf_cap = final_user_rf * 0.90; g_config_cap = TERMINAL_GROWTH_RATE_CAP
    projection_log.append(f"\nDetermining Terminal Growth (g)..."); projection_log.append(f"  Caps: Config={format_percent(g_config_cap)}, WACC*0.6={format_percent(g_wacc_cap)}, Initial={format_percent(g_init_final)}, Rf*0.9={format_percent(g_rf_cap)}")
    valid_caps = [cap for cap in [g_config_cap, g_wacc_cap, g_init_final, g_rf_cap] if np.isfinite(cap)]
    if not valid_caps: term_g = 0.0; projection_log.append("  ⚠️ No valid caps. Using 0%.")
    else: term_g = min(valid_caps); projection_log.append(f"  ↳ Min Cap Applied: {format_percent(term_g)}")
    original_term_g = term_g; term_g = max(MIN_GROWTH, term_g)
    if abs(term_g - original_term_g) > 1e-6: projection_log.append(f"  ↳ Floor Applied ({format_percent(MIN_GROWTH)}): {format_percent(term_g)}")
    if term_g >= disc_rate: original_term_g = term_g; term_g = max(disc_rate - 0.005, MIN_GROWTH); projection_log.append(f"  ⚠️ Terminal Growth ({format_percent(original_term_g)}) was >= WACC ({format_percent(disc_rate)}). Reduced to {format_percent(term_g)}.")
    projection_log.append(f"  ▶️ Final Terminal Growth Rate (g): {format_percent(term_g)}"); projection_results['terminal_growth_rate'] = term_g

    # Project FCFF for forecast_yrs (e.g., 3 years)
    projected_fcffs = []; growth_rates_used = []; pv_fcff_list = []; cumulative_pv = 0.0; last_fcff = precalculated_avg_basis
    eff_init_g = init_growth if pd.notna(init_growth) and math.isfinite(init_growth) else 0.0; eff_term_g = term_g if pd.notna(term_g) and math.isfinite(term_g) else 0.0
    projection_log.append(f"\n--- FCFF Projections ({forecast_yrs} Years) ---"); projection_log.append(f"Basis (T=0 FCFF): {format_currency(last_fcff)}")
    header = f"{'Year':<5} {'Growth Rate':<15} {'Projected FCFF':<25} {'PV Factor':<15} {'PV of FCFF':<25}"; projection_log.append(header); projection_log.append("-" * len(header))
    for yr in range(1, forecast_yrs + 1):
        decay_factor = max(0, (forecast_yrs - yr)) / forecast_yrs if forecast_yrs > 0 else 0; current_growth = eff_term_g + (eff_init_g - eff_term_g) * decay_factor; current_growth = max(MIN_GROWTH, current_growth); growth_rates_used.append(current_growth)
        fcff_proj = last_fcff * (1 + current_growth)
        if not np.isfinite(fcff_proj) or abs(fcff_proj) > 1e18: fcff_proj = 0.0; projection_log.append(f"⚠️ Yr {yr} Projected FCFF ({fcff_proj}) invalid. Setting to 0.")
        projected_fcffs.append(fcff_proj); last_fcff = fcff_proj
        pv_factor = 1 / ((1 + disc_rate)**yr); pv_fcff_val = fcff_proj * pv_factor
        if not np.isfinite(pv_fcff_val): pv_fcff_val = 0.0
        pv_fcff_list.append(pv_fcff_val); cumulative_pv += pv_fcff_val
        log_line = f"{yr:<5} {format_percent(current_growth):<15} {format_currency(fcff_proj):<25} {pv_factor:<15.4f} {format_currency(pv_fcff_val):<25}"; projection_log.append(log_line)
    projection_results['projected_fcffs'] = projected_fcffs; projection_results['growth_rates'] = growth_rates_used; projection_results['pv_fcffs'] = pv_fcff_list
    projection_results['cumulative_pv_fcff'] = cumulative_pv if np.isfinite(cumulative_pv) else 0.0
    projection_log.append("-" * len(header)); projection_log.append(f"Sum of Present Values of FCFFs (Years 1-{forecast_yrs}): {format_currency(projection_results['cumulative_pv_fcff'])}")

    # Calculate Terminal Value (TV)
    projection_log.append("\n--- Terminal Value Calculation ---")
    terminal_value = np.nan; pv_terminal_value = np.nan
    final_projected_fcff = projected_fcffs[-1] if projected_fcffs else np.nan # FCFF in the last forecast year (Year N)

    # Check conditions for Gordon Growth Model
    denominator = disc_rate - eff_term_g
    if denominator <= 1e-6: # WACC must be greater than terminal growth rate
        projection_results['error'] = f"WACC ({format_percent(disc_rate)}) <= Terminal Growth ({format_percent(eff_term_g)}). Cannot calculate Terminal Value."
        terminal_value = 0.0; pv_terminal_value = 0.0;
        projection_log.append(f"❌ ERROR: {projection_results['error']}. Setting TV and PV(TV) to 0.")
    elif not np.isfinite(final_projected_fcff): # Need the last projected FCFF
        projection_results['error'] = "Final projected FCFF is invalid. Cannot calculate Terminal Value."
        terminal_value = 0.0; pv_terminal_value = 0.0;
        projection_log.append(f"❌ ERROR: {projection_results['error']}. Setting TV and PV(TV) to 0.")
    else:
         # Calculate FCFF in Year N+1
         fcff_n_plus_1 = final_projected_fcff * (1 + eff_term_g)
         projection_log.append(f"FCFF in Year {forecast_yrs} (Final Forecast Year): {format_currency(final_projected_fcff)}")
         projection_log.append(f"Projected FCFF in Year {forecast_yrs + 1} (for TV Calc): {format_currency(fcff_n_plus_1)}")

         # Calculate Terminal Value at Year N
         terminal_value = fcff_n_plus_1 / denominator

         # Check if TV calculation resulted in non-finite or negative value
         # *** SYNTAX FIX APPLIED HERE V21.7.1 ***
         if not np.isfinite(terminal_value) or terminal_value < 0:
             projection_log.append(f"⚠️ Warning: Calculated Terminal Value is invalid or negative ({format_currency(terminal_value)}). Setting TV and PV(TV) to 0.")
             # Only set error if not already set by a previous fatal condition (like WACC<=g)
             if projection_results['error'] is None:
                 projection_results['error'] = "Calculated Terminal Value invalid/negative."
             terminal_value = 0.0
             pv_terminal_value = 0.0
         else:
             # This part only runs if TV is finite and non-negative
             projection_log.append(f"  Calculated Terminal Value (Year {forecast_yrs}): {format_currency(terminal_value)}")
             # Calculate Present Value of Terminal Value
             pv_factor_tv = 1 / ((1 + disc_rate)**forecast_yrs)
             pv_terminal_value = terminal_value * pv_factor_tv
             if not np.isfinite(pv_terminal_value): # Handle PV calc failure
                 pv_terminal_value = 0.0
                 projection_log.append(f"⚠️ Warning: PV(TV) calculation failed. Setting to 0.")
                 if projection_results['error'] is None: projection_results['error']="PV(TV) calculation failed."

             projection_log.append(f"Present Value Factor for TV (Year {forecast_yrs}): {pv_factor_tv:.4f}")
             projection_log.append(f"Present Value of Terminal Value (PV(TV)): {format_currency(pv_terminal_value)}")

    # Store results
    projection_results['terminal_value'] = terminal_value if np.isfinite(terminal_value) else 0.0
    projection_results['pv_terminal_value'] = pv_terminal_value if np.isfinite(pv_terminal_value) else 0.0

    # Display the log
    display(Markdown("```\n"+"\n".join(projection_log)+"\n```"))

    return projection_results

# --- Execute FCFF calculation and projection ---
fcff_results = {'error': 'FCFF Calculation not run.', 'initial_growth_rate': np.nan, 'growth_source':'N/A', 'terminal_growth_rate': np.nan, 'fcff_basis_method':'N/A'}

if selected_fcff_method is not None and np.isfinite(discount_rate):
    avg_basis_chosen = precalculated_averages.get(selected_fcff_method, np.nan)
    if np.isfinite(avg_basis_chosen) and avg_basis_chosen > 1e-9:
        try:
            # Call the updated V21.7 function
            fcff_results = calculate_and_project_fcff_user_selected_method_v21_7(
                user_selected_method_idx=selected_fcff_method,
                precalculated_avg_basis=avg_basis_chosen,
                financials_df=stock_data.get('financials'),
                balance_sheet_df=stock_data.get('balance_sheet'),
                cashflow_df=stock_data.get('cashflow'),
                wacc_comps=wacc_components,
                disc_rate=discount_rate,
                forecast_yrs=FORECAST_YEARS,
                final_user_rf=risk_free_rate_input,
                default_init_growth=DEFAULT_GROWTH_RATE,
                currency_sym=currency_symbol
            )
        except Exception as e:
             display(Markdown(f"❌ **CRITICAL ERROR during FCFF Projection function call: {e}**")); print(traceback.format_exc())
             fcff_results={'error': f'Runtime error during projection: {e}'}
    else:
        display(Markdown(f"❌ **Skipping FCFF Projection:** Basis for selected method {selected_fcff_method} invalid ({format_currency(avg_basis_chosen)})."))
        fcff_results={'error': f'Invalid/non-positive basis for method {selected_fcff_method}. Cannot proceed.'}
elif not np.isfinite(discount_rate):
    display(Markdown("❌ **Skipping FCFF Projection:** Invalid WACC."))
    fcff_results={'error': 'Invalid WACC prevents projection.'}
elif selected_fcff_method is None:
    display(Markdown("❌ **Skipping FCFF Projection:** No valid FCFF method selected."))
    fcff_results={'error': 'No valid FCFF method selected. Cannot proceed.'}


# %% [markdown]
# ## 8. Enterprise Value and Equity Value Calculation (Updated V21.7.2 - Bridge Syntax Fix)

# %%
# Valuation Function Definition (V21.7.2 Bridge Syntax Fix)
def calculate_final_valuation(fcff_data, balance_sheet_df, shares_outstanding_val, currency_sym, wacc_data):
    display(Markdown("### Final Valuation: Enterprise Value to Equity Value Bridge"))
    valuation = {'enterprise_value': np.nan, 'equity_value': np.nan, 'intrinsic_value_per_share': np.nan, 'debt_used': np.nan, 'cash_used': np.nan, 'minority_interest_used': np.nan, 'error': None}; val_log = []
    if not isinstance(fcff_data, dict):
        valuation['error'] = "FCFF Data invalid."; val_log.append(f"❌ FATAL ERROR: {valuation['error']}"); display(Markdown("```\n"+"\n".join(val_log)+"\n```")); return valuation
    fcff_error = fcff_data.get('error'); is_fcff_fatal = fcff_error and ("Cannot proceed" in fcff_error or "invalid basis" in fcff_error or "No valid FCFF method" in fcff_error or "WACC <=" in fcff_error or "Invalid WACC" in fcff_error)
    if is_fcff_fatal:
        valuation['error'] = f"Valuation halted by FCFF error: {fcff_error}"; val_log.append(f"❌ FATAL ERROR: {valuation['error']}"); display(Markdown("```\n"+"\n".join(val_log)+"\n```")); return valuation
    elif fcff_error: val_log.append(f"ℹ️ Note: Proceeding despite FCFF warning: {fcff_error}")
    pv_fcff = fcff_data.get('cumulative_pv_fcff', np.nan); pv_tv = fcff_data.get('pv_terminal_value', np.nan)
    if not np.isfinite(pv_fcff): pv_fcff = 0.0; val_log.append(f"⚠️ PV(FCFFs) invalid, assuming 0.")
    if not np.isfinite(pv_tv): pv_tv = 0.0; val_log.append(f"⚠️ PV(TV) invalid, assuming 0.")
    enterprise_value = pv_fcff + pv_tv; valuation['enterprise_value'] = enterprise_value
    val_log.append(f"Sum of PV(Forecast FCFFs): {format_currency(pv_fcff)}"); val_log.append(f"PV(Terminal Value): {format_currency(pv_tv)}"); val_log.append(f"↳ Calculated Enterprise Value (EV): {format_currency(enterprise_value)}")

    # --- Bridge EV to Equity Value ---
    debt = 0.0; cash = 0.0; minority_interest = 0.0
    debt_src = 'N/A'; cash_src = 'N/A'; mi_src = 'N/A'
    # Get Debt used in WACC calculation
    if isinstance(wacc_data, dict):
        debt = wacc_data.get('debt_value_used', 0.0)
        debt_src = wacc_data.get('debt_calc_method', '?')
        if not np.isfinite(debt):
             val_log.append(f"⚠️ Debt value from WACC components invalid ({debt}). Assuming 0.")
             debt = 0.0
             debt_src += " (Invalid->0)"
        valuation['debt_used'] = debt
    # *** SYNTAX FIX APPLIED HERE V21.7.2 ***
    else:
        val_log.append(f"⚠️ WACC components data invalid. Assuming 0 Debt for bridge.")
        debt_src = "WACC Data Missing"
        valuation['debt_used'] = 0.0
        # Note: debt variable is already 0.0 by default if wacc_data isn't a dict

    # Get Cash and Minority Interest from latest Balance Sheet
    if balance_sheet_df is not None and not balance_sheet_df.empty:
        bs_latest = balance_sheet_df.iloc[:, 0]; val_log.append(f"\nBridging EV to Equity Value:")
        cash_keys = ["Cash And Cash Equivalents", "Cash", "Cash Cash Equivalents And Short Term Investments"]; cash = safe_get(bs_latest, cash_keys, "BS(Cash)", 0)
        if not np.isfinite(cash): val_log.append(f"⚠️ Cash invalid. Assuming 0."); cash = 0.0; cash_src = "BS Invalid->0"
        else: cash_src = "BS Latest"
        valuation['cash_used'] = cash
        mi_keys = ["Minority Interest", "Noncontrolling Interest In Consolidated Entity"]; minority_interest = safe_get(bs_latest, mi_keys, "BS(MI)", 0)
        if not np.isfinite(minority_interest): minority_interest = 0.0; mi_src = "BS Invalid->0"
        else: mi_src = "BS Latest"
        valuation['minority_interest_used'] = minority_interest
        val_log.append(f"  - Total Debt ({debt_src}): {format_currency(debt)}"); val_log.append(f"  + Cash & Equivalents ({cash_src}): {format_currency(cash)}")
        if abs(minority_interest) > 1e-6: val_log.append(f"  - Minority Interest ({mi_src}): {format_currency(minority_interest)}")
        else: val_log.append(f"  - Minority Interest ({mi_src}): Approx Zero")
    else: val_log.append("⚠️ BS missing. Cannot fetch Cash/MI."); val_log.append(f"\nBridging EV to Equity (Assumptions):"); val_log.append(f"  - Debt ({debt_src}): {format_currency(debt)}"); val_log.append(f"  + Cash: {format_currency(cash)}"); val_log.append(f"  - MI: {format_currency(minority_interest)}"); cash_src = "Assumed 0"; mi_src = "Assumed 0"; valuation['cash_used'] = cash; valuation['minority_interest_used'] = minority_interest
    if not all(np.isfinite(v) for v in [enterprise_value, debt, cash, minority_interest]):
        valuation['error'] = "Invalid bridge component."; val_log.append(f"❌ ERROR: {valuation['error']}"); valuation['equity_value'] = np.nan; valuation['intrinsic_value_per_share'] = np.nan; display(Markdown("```\n"+"\n".join(val_log)+"\n```")); return valuation
    equity_value = enterprise_value - debt + cash - minority_interest; valuation['equity_value'] = equity_value; val_log.append(f"↳ Calculated Equity Value: {format_currency(equity_value)}")
    intrinsic_value_per_share = np.nan
    if not np.isfinite(equity_value): valuation['error'] = f"Equity val invalid."; val_log.append(f"❌ ERROR: {valuation['error']}")
    elif not np.isfinite(shares_outstanding_val) or shares_outstanding_val <= 0: valuation['error'] = f"Shares invalid."; val_log.append(f"❌ ERROR: {valuation['error']}")
    else: intrinsic_value_per_share = equity_value / shares_outstanding_val; val_log.append(f"\nIVPS:"); val_log.append(f"  Equity Value: {format_currency(equity_value)}"); val_log.append(f"  ÷ Shares: {shares_outstanding_val:,.0f}"); val_log.append(f"▶️ IV/Share: {format_currency(intrinsic_value_per_share)}")
    valuation['intrinsic_value_per_share'] = intrinsic_value_per_share if np.isfinite(intrinsic_value_per_share) else np.nan
    display(Markdown("```\n"+"\n".join(val_log)+"\n```")); return valuation

# --- Execute Final Valuation ---
final_valuation = {'error': 'Final Valuation calc not run.', 'intrinsic_value_per_share': np.nan}
fcff_ready = isinstance(fcff_results, dict); wacc_ready = isinstance(wacc_components, dict)
fcff_fatal_error = False
if fcff_ready:
    fcff_error_msg = fcff_results.get('error', '');
    if fcff_error_msg and ("Cannot proceed" in fcff_error_msg or "invalid basis" in fcff_error_msg or "No valid FCFF method" in fcff_error_msg or "WACC <=" in fcff_error_msg or "Invalid WACC" in fcff_error_msg):
        fcff_fatal_error = True; final_valuation['error'] = f"Valuation halted by FCFF error: {fcff_error_msg}"
if fcff_ready and wacc_ready and not fcff_fatal_error:
    try: final_valuation = calculate_final_valuation(fcff_data=fcff_results, balance_sheet_df=stock_data.get('balance_sheet'), shares_outstanding_val=shares_outstanding, currency_sym=currency_symbol, wacc_data=wacc_components)
    except Exception as e: display(Markdown(f"❌ **CRITICAL ERROR Final Val: {e}**")); print(traceback.format_exc()); final_valuation = {'error': f'Runtime error: {e}', 'intrinsic_value_per_share': np.nan}
elif fcff_fatal_error: display(Markdown(f"❌ **Skipping Final Valuation:** {final_valuation['error']}"))
else: missing_inputs = [];
if not fcff_ready: missing_inputs.append("FCFF Results");
if not wacc_ready: missing_inputs.append("WACC Components"); err_msg = f"Invalid input data ({', '.join(missing_inputs)})."; display(Markdown(f"❌ **Skipping Final Val:** {err_msg}")); final_valuation = {'error': f'Upstream data error: {err_msg}', 'intrinsic_value_per_share': np.nan}


# %% [markdown]
# ## 9. Financial Ratio Analysis (Updated V21.7.3 - safe_div Syntax Fixed)

# %%
# Ratio Function Definition (V21.7.3 safe_div Syntax Fix)
def analyze_ratios(stock_info_dict, financials_df, balance_sheet_df, cashflow_df,
                   enterprise_val, current_price_val, shares_outstanding_val,
                   currency_sym, user_benchmark_dict, benchmark_source_dict, wacc_data):

    display(Markdown("### Financial Ratio Analysis..."))
    ratios = {}; ratio_log = []; ratio_data_for_table = []
    ratio_plotly_data = {'Ratio': [], 'Company Value': [], 'Benchmark Value': [], 'Benchmark Source': [], 'Comparison Status': []}
    company_nm = stock_info_dict.get("shortName", ticker_symbol)
    valid_price = current_price_val is not None and np.isfinite(current_price_val) and current_price_val > 0
    valid_shares = shares_outstanding_val is not None and np.isfinite(shares_outstanding_val) and shares_outstanding_val > 0
    valid_ev = enterprise_val is not None and np.isfinite(enterprise_val)
    if not valid_price: ratio_log.append("⚠️ Price invalid. P/E, P/B N/A.")
    if not valid_shares: ratio_log.append("⚠️ Shares invalid. Per-share ratios N/A.")
    if not valid_ev: ratio_log.append("⚠️ EV invalid. EV/EBITDA N/A.")

    essential_missing_internal = False
    if financials_df is None or financials_df.empty: ratio_log.append("❌ ERROR (Internal): IS missing."); essential_missing_internal = True
    if balance_sheet_df is None or balance_sheet_df.empty: ratio_log.append("❌ ERROR (Internal): BS missing."); essential_missing_internal = True
    if cashflow_df is None or cashflow_df.empty: ratio_log.append("⚠️ Warning (Internal): CF missing. D&A fallback attempted.")
    if essential_missing_internal: display(Markdown("```\n" + "\n".join(ratio_log) + "\n```")); display(Markdown("*Cannot proceed with ratios (missing essential statements).*")); return {}, []

    try:
        latest_fin = financials_df.iloc[:, 0]; latest_bs = balance_sheet_df.iloc[:, 0]
        latest_cf = cashflow_df.iloc[:, 0] if cashflow_df is not None and not cashflow_df.empty else pd.Series(dtype=float)
    except IndexError: ratio_log.append("❌ ERROR: Cannot access latest statement column."); display(Markdown("```\n" + "\n".join(ratio_log) + "\n```")); return {}, []

    ratio_log.append("--- Calculating Ratio Components ---")
    net_income = safe_get(latest_fin, ["Net Income", "Net Income Common Stockholders"], "IS(Net Income)", default=np.nan)
    revenue = safe_get(latest_fin, ["Total Revenue", "Revenue"], "IS(Revenue)", default=np.nan)
    ebit = safe_get(latest_fin, ["EBIT", "Ebit", "Operating Income"], "IS(EBIT)", default=np.nan)
    ratio_log.append(f"  Net Income: {format_currency(net_income)}, Revenue: {format_currency(revenue)}, EBIT: {format_currency(ebit)}")
    dep_keys = ["Depreciation And Amortization", "Depreciation"]; dep = safe_get(latest_cf, dep_keys, "CF(Depr)", default=np.nan); dep_source = "CF"
    if not np.isfinite(dep):
        dep_is = safe_get(latest_fin, dep_keys, "IS(Depr Fallback)", default=np.nan)
        if np.isfinite(dep_is): dep = dep_is; dep_source = "IS Fallback"; ratio_log.append(f"  Depr (from IS): {format_currency(dep)}")
        else: dep = 0.0; dep_source = "Assumed 0"; ratio_log.append(f"  ⚠️ Depr not found. Assuming 0.")
    else: ratio_log.append(f"  Depr (from CF): {format_currency(dep)}")
    dep = float(dep)
    ebitda = np.nan
    if np.isfinite(ebit) and np.isfinite(dep): ebitda = ebit + dep; ratio_log.append(f"  EBITDA: {format_currency(ebitda)}")
    elif np.isfinite(ebit): ebitda = ebit; ratio_log.append(f"  ℹ️ EBITDA=EBIT (Depr was {dep_source}): {format_currency(ebitda)}")
    else: ratio_log.append(f"  ⚠️ Cannot calc EBITDA.")
    equity = safe_get(latest_bs, ["Total Stockholder Equity", "Common Stock Equity", "Stockholders Equity"], "BS(Equity)", default=np.nan)
    assets = safe_get(latest_bs, ["Total Assets"], "BS(Assets)", default=np.nan)
    curr_a = safe_get(latest_bs, ['Current Assets', 'Total Current Assets'], "BS(Current Assets)", default=np.nan)
    curr_l = safe_get(latest_bs, ['Current Liabilities', 'Total Current Liabilities'], "BS(Current Liab)", default=np.nan)
    inv = safe_get(latest_bs, ["Inventory"], "BS(Inventory)", default=np.nan)
    ratio_log.append(f"  Equity: {format_currency(equity)}, Assets: {format_currency(assets)}"); ratio_log.append(f"  Curr Assets: {format_currency(curr_a)}, Curr Liab: {format_currency(curr_l)}")
    if not np.isfinite(inv): inv = 0.0; ratio_log.append(f"  ℹ️ Inventory N/A. Assuming 0.")
    else: ratio_log.append(f"  Inventory: {format_currency(inv)}")
    inv = float(inv)
    debt = 0.0; debt_src = 'N/A';
    if isinstance(wacc_data, dict): debt = wacc_data.get('debt_value_used', 0.0); debt_src = wacc_data.get('debt_calc_method', '?');
    if not np.isfinite(debt): debt = 0.0; debt_src = f"WACC Calc ({debt_src})"
    else: debt = 0.0; debt_src = "WACC Data Missing (Assumed 0)"; ratio_log.append(f"⚠️ Assuming 0 Debt.")
    ratio_log.append(f"  Total Debt (from {debt_src}): {format_currency(debt)}")
    eps = (net_income / shares_outstanding_val) if valid_shares and np.isfinite(net_income) and shares_outstanding_val > 0 else np.nan
    bvps = (equity / shares_outstanding_val) if valid_shares and np.isfinite(equity) and shares_outstanding_val > 0 else np.nan
    ratio_log.append(f"  EPS: {format_currency(eps) if np.isfinite(eps) else 'N/A'}, BVPS: {format_currency(bvps) if np.isfinite(bvps) else 'N/A'}")
    cap_emp = np.nan; ce_src = "N/A"
    if np.isfinite(assets) and np.isfinite(curr_l): cap_emp_m1 = assets - curr_l;
    if np.isfinite(cap_emp_m1) and cap_emp_m1 > 1.0: cap_emp = cap_emp_m1; ce_src = "A-CL"; ratio_log.append(f"  Cap Employed ({ce_src}): {format_currency(cap_emp)}")
    if not np.isfinite(cap_emp):
        ratio_log.append(f"  Cap Emp (A-CL) failed. Trying E+D...")
        if np.isfinite(equity) and np.isfinite(debt): cap_emp_m2 = equity + debt;
        if np.isfinite(cap_emp_m2) and cap_emp_m2 > 1.0: cap_emp = cap_emp_m2; ce_src = "E+D"; ratio_log.append(f"  Cap Employed ({ce_src}): {format_currency(cap_emp)}")
        else: ratio_log.append(f"  Cap Emp (E+D) failed.")
    if not np.isfinite(cap_emp): ce_src = "Failed"; ratio_log.append("⚠️ Cap Emp failed. ROCE N/A.")

    ratio_log.append("\n--- Calculating Final Ratios ---")

    # *** SYNTAX FIX APPLIED HERE V21.7.3 ***
    # Helper for safe division (Corrected V21.7.3)
    def safe_div(numerator, denominator):
        # Attempt to convert inputs to float, handle non-finite values
        try:
            num = float(numerator)
            if not np.isfinite(num):
                num = np.nan # Treat non-finite numerators as NaN
        except (ValueError, TypeError):
            num = np.nan # Treat conversion errors as NaN

        try:
            den = float(denominator)
            if not np.isfinite(den):
                den = np.nan # Treat non-finite denominators as NaN
        except (ValueError, TypeError):
            den = np.nan # Treat conversion errors as NaN

        # Check for invalid conditions: NaN inputs or near-zero denominator
        if pd.isna(num) or pd.isna(den) or abs(den) < 1e-9:
            return np.nan
        else:
            # Perform division only if inputs are valid and denominator is safe
            try:
                result = num / den
                # Optional: Check if result itself is finite (e.g., avoid inf/inf)
                if not np.isfinite(result):
                    return np.nan
                return result
            except ZeroDivisionError: # Should be caught by abs(den) check, but good practice
                return np.nan
            except Exception: # Catch any other potential math errors
                return np.nan

    # Calculate Ratios using the corrected safe_div
    ratios['P/E'] = safe_div(current_price_val, eps) if valid_price else np.nan
    ratios['P/B'] = safe_div(current_price_val, bvps) if valid_price else np.nan
    ratios['EV/EBITDA'] = safe_div(enterprise_val, ebitda) if valid_ev and np.isfinite(ebitda) else np.nan
    ratios['ROE'] = safe_div(net_income, equity); ratios['ROA'] = safe_div(net_income, assets); ratios['ROCE'] = safe_div(ebit, cap_emp)
    ratios['Asset Turnover'] = safe_div(revenue, assets); ratios['Current Ratio'] = safe_div(curr_a, curr_l)
    quick_assets = (curr_a - inv) if np.isfinite(curr_a) and np.isfinite(inv) else np.nan; ratios['Quick Ratio'] = safe_div(quick_assets, curr_l)
    for key, val in ratios.items(): is_perc = key in ['ROE','ROA','ROCE']; fmt_val = format_percent(val, 1) if is_perc else (f"{val:.2f}" if np.isfinite(val) else "N/A"); ratio_log.append(f"  {key}: {fmt_val}")

    # (Rest of the ratio comparison and plotting logic remains the same as V21.7.2)
    display(Markdown("*(Comparing ratios vs benchmarks...)*"))
    for key in REQUIRED_BENCHMARK_RATIOS:
        comp_val = ratios.get(key); is_perc = key in ['ROE','ROA','ROCE']
        val_fmt = "N/A";
        if pd.notna(comp_val) and math.isfinite(comp_val): val_fmt = format_percent(comp_val, 1) if is_perc else f"{comp_val:.2f}"
        elif pd.notna(comp_val): val_fmt = "Infinity" if comp_val > 0 else "-Infinity"
        bench_val = user_benchmark_dict.get(key, np.nan); b_src = benchmark_source_dict.get(key, "?")
        bench_fmt = "N/A";
        if np.isfinite(bench_val): bench_fmt = format_percent(bench_val, 1) if is_perc else f"{bench_val:.2f}"; bench_fmt += f" <small><i>({b_src.replace(' Input','').replace('Fallback ','')})</i></small>"
        else: bench_fmt = "<i style='color:grey'>N/A</i>"
        status = "<i style='color:grey'>N/A</i>"; status_color = "grey"; plot_comp_val = np.nan; plot_bench_val = np.nan
        if np.isfinite(comp_val) and np.isfinite(bench_val):
            num_val = comp_val * 100 if is_perc else comp_val; num_bench = bench_val * 100 if is_perc else bench_val
            plot_comp_val = num_val; plot_bench_val = num_bench
            good_color = '#90EE90'; bad_color = '#F08080'; neutral_color = '#FFD700'; avg_color = '#D3D3D3'
            if abs(num_bench) > 1e-9: ratio_diff = (num_val - num_bench) / abs(num_bench)
            else: ratio_diff = float('inf') if num_val > 0 else float('-inf') if num_val < 0 else 0.0
            if key in ['ROE','ROA','ROCE','Asset Turnover','Current Ratio','Quick Ratio']:
                 if ratio_diff > 0.20: status_color = good_color; status_text = "⬆️ Stronger"
                 elif ratio_diff < -0.20: status_color = bad_color; status_text = "⬇️ Weaker"
                 else: status_color = avg_color; status_text = "↔️ Average"
            elif key in ['P/E','P/B','EV/EBITDA']:
                 if ratio_diff < -0.20: status_color = good_color; status_text = "✅ Lower (Favorable)"
                 elif ratio_diff > 0.50: status_color = bad_color; status_text = "‼️ Higher (Unfavorable)"
                 elif ratio_diff > 0.20: status_color = neutral_color; status_text = "⚠️ Elevated"
                 else: status_color = avg_color; status_text = "↔️ Average"
            else: status_text = "~ N/A ~"
            status = f"<span style='color:{status_color}; font-weight:bold;'>{status_text}</span>"
            ratio_plotly_data['Ratio'].append(key); ratio_plotly_data['Company Value'].append(plot_comp_val); ratio_plotly_data['Benchmark Value'].append(plot_bench_val); ratio_plotly_data['Benchmark Source'].append(b_src); ratio_plotly_data['Comparison Status'].append(status_text)
        elif not np.isfinite(comp_val): status = "<i style='color:grey'>N/A (Calc Fail)</i>"; ratio_plotly_data['Ratio'].append(key); ratio_plotly_data['Company Value'].append(np.nan); ratio_plotly_data['Benchmark Value'].append(bench_val * 100 if is_perc and np.isfinite(bench_val) else bench_val); ratio_plotly_data['Benchmark Source'].append(b_src); ratio_plotly_data['Comparison Status'].append("N/A (Calc Fail)")
        elif not np.isfinite(bench_val): status = "<i style='color:grey'>N/A (No Bench)</i>"; ratio_plotly_data['Ratio'].append(key); ratio_plotly_data['Company Value'].append(comp_val * 100 if is_perc else comp_val); ratio_plotly_data['Benchmark Value'].append(np.nan); ratio_plotly_data['Benchmark Source'].append(b_src); ratio_plotly_data['Comparison Status'].append("N/A (No Bench)")
        ratio_data_for_table.append([f"<b>{key}</b>", val_fmt, bench_fmt, status])

    if ratio_log: display(Markdown("```\n" + "\n".join(ratio_log) + "\n```"))
    if ratio_data_for_table:
        header = ["Ratio", f"{company_nm} Value", "Benchmark", "Status vs Benchmark"]; html = "<p style='font-weight:bold;'>Ratio Comparison Summary:</p><table border='1' style='width:98%; font-size:0.9em; border-collapse: collapse;'><thead style='background-color:#e9ecef;'><tr>" + "".join(f"<th style='padding: 5px; text-align: left;'>{h}</th>" for h in header) + "</tr></thead><tbody>"
        for i, row in enumerate(ratio_data_for_table): bg_color = "#f8f9fa" if i % 2 == 0 else "#ffffff"; html += f"<tr style='background-color:{bg_color};'><td style='padding: 5px;'>{row[0]}</td><td style='padding: 5px; text-align: right;'>{row[1]}</td><td style='padding: 5px; text-align: right;'>{row[2]}</td><td style='padding: 5px; text-align: center;'>{row[3]}</td></tr>"
        html += "</tbody></table><br>"; display(HTML(html))
    else: display(Markdown("*No ratio data for table.*"))

    if ratio_plotly_data['Ratio'] and any(np.isfinite(v) for v in ratio_plotly_data['Company Value']) and any(np.isfinite(v) for v in ratio_plotly_data['Benchmark Value']):
       valid_plot_indices = [i for i, (cv, bv) in enumerate(zip(ratio_plotly_data['Company Value'], ratio_plotly_data['Benchmark Value'])) if np.isfinite(cv) and np.isfinite(bv)]
       if not valid_plot_indices: display(Markdown("*No ratios with valid Company and Benchmark values found for plotting.*"))
       else:
            num_ratios_to_plot = len(valid_plot_indices)
            specs_list = [[{'type': 'indicator'}]] * num_ratios_to_plot
            fig_ratios = make_subplots(rows=num_ratios_to_plot, cols=1, specs=specs_list, subplot_titles=[ratio_plotly_data['Ratio'][i] for i in valid_plot_indices], vertical_spacing=0.05 + 0.05/num_ratios_to_plot)

            def get_range_and_delta_colors(ratio_key):
                good_color = '#90EE90'; bad_color = '#F08080'; neutral_color = '#FFD700'; avg_color = '#D3D3D3'
                if ratio_key in ['ROE','ROA','ROCE','Asset Turnover','Current Ratio','Quick Ratio']: colors = [bad_color, avg_color, good_color]; decreasing_color = bad_color; increasing_color = good_color; delta_threshold = 0.20
                elif ratio_key in ['P/E','P/B','EV/EBITDA']: colors = [good_color, avg_color, bad_color]; decreasing_color = good_color; increasing_color = bad_color; delta_threshold = 0.20
                else: colors = ['lightgrey','darkgrey','grey']; decreasing_color = 'grey'; increasing_color = 'grey'; delta_threshold = 0.20
                return colors, decreasing_color, increasing_color, delta_threshold

            plot_row = 1
            for i in valid_plot_indices:
                name = ratio_plotly_data['Ratio'][i]; comp_v = ratio_plotly_data['Company Value'][i]; bench_v = ratio_plotly_data['Benchmark Value'][i]; b_src_p = ratio_plotly_data['Benchmark Source'][i]; is_perc = name in ['ROE','ROA','ROCE']
                range_colors, dec_color, inc_color, delta_thresh = get_range_and_delta_colors(name)
                val_range = abs(comp_v - bench_v); padding = max(val_range * 0.5, abs(bench_v * 0.3), 5 if is_perc else 1); gauge_min = min(comp_v, bench_v) - padding; gauge_max = max(comp_v, bench_v) + padding
                if name in ['P/E','EV/EBITDA']: gauge_min = max(gauge_min, -20); gauge_max = min(gauge_max, 100)
                elif name == 'P/B': gauge_min = max(gauge_min, -5); gauge_max = min(gauge_max, 20)
                elif name in ['Current Ratio','Quick Ratio']: gauge_min = max(gauge_min, 0); gauge_max = min(gauge_max, 5)
                elif is_perc: gauge_min = max(gauge_min, -100); gauge_max = min(gauge_max, 150)
                elif name == 'Asset Turnover': gauge_min = max(gauge_min, -0.5); gauge_max = min(gauge_max, 3)
                if gauge_min >= gauge_max: gauge_max = gauge_min + max(1, abs(gauge_min * 0.1))
                step_delta = abs(bench_v * delta_thresh) if abs(bench_v)>1e-6 else (1 if is_perc else 0.1); bound1 = bench_v - step_delta; bound2 = bench_v + step_delta
                steps_config = [{'range': [gauge_min, bound1], 'color': range_colors[0]}, {'range': [bound1, bound2], 'color': range_colors[1]}, {'range': [bound2, gauge_max], 'color': range_colors[2]}]
                steps_config = [s for s in steps_config if s['range'][0] < s['range'][1]];
                if not steps_config: steps_config = [{'range': [gauge_min, gauge_max], 'color':'lightgrey'}]
                bench_fmt_plot = (f"{bench_v:.1f}%" if is_perc else f"{bench_v:.2f}") + f" <small><i>({b_src_p.replace('Fallback ','')})</i></small>"; indicator_title = f"<i style='font-size:0.8em'>(Benchmark: {bench_fmt_plot})</i>"; value_format = ".1f" if is_perc else ".2f"
                fig_ratios.add_trace(go.Indicator(mode="number+gauge+delta", value=comp_v, number={'suffix': "%" if is_perc else "", 'valueformat': value_format}, delta={'reference': bench_v, 'relative': False, 'valueformat': value_format, 'decreasing': {'color': dec_color}, 'increasing': {'color': inc_color}}, domain={'row': plot_row - 1, 'column': 0}, title={'text': indicator_title, 'font': {'size': 10}}, gauge={'shape': "bullet", 'axis': {'range': [gauge_min, gauge_max], 'tickformat': value_format}, 'threshold': {'line': {'color': "black", 'width': 2}, 'thickness': 0.75, 'value': bench_v}, 'steps': steps_config, 'bar': {'color': "#2c3e50"}}), row=plot_row, col=1)
                plot_row += 1
            fig_ratios.update_layout(height=max(300, num_ratios_to_plot * 85), title_text='Ratio Performance vs. Benchmarks', title_x=0.5, margin=dict(l=180, r=40, t=60 + num_ratios_to_plot * 15 , b=30))
            fig_ratios.show()
    else: display(Markdown("*No comparable ratio data for chart.*"))
    return ratios, ratio_data_for_table

# --- Execute Ratio Analysis ---
ratios_calculated = {}; ratio_table = []
valuation_results_exist = isinstance(final_valuation, dict); wacc_components_exist = isinstance(wacc_components, dict)
financials_present = stock_data.get('financials') is not None and not stock_data['financials'].empty
balance_sheet_present = stock_data.get('balance_sheet') is not None and not stock_data['balance_sheet'].empty
required_statements_present = financials_present and balance_sheet_present
valuation_ok_for_ratios = valuation_results_exist and not (final_valuation.get('error') and "halted" in final_valuation.get('error',''))

if valuation_ok_for_ratios and wacc_components_exist and required_statements_present:
    try:
        ev_for_ratios = final_valuation.get('enterprise_value', np.nan)
        if not np.isfinite(ev_for_ratios): print("INFO (Pre-call): EV is invalid/NaN. EV/EBITDA N/A.")
        ratios_calculated, ratio_table = analyze_ratios(stock_info_dict=stock_data['info'], financials_df=stock_data.get('financials'), balance_sheet_df=stock_data.get('balance_sheet'), cashflow_df=stock_data.get('cashflow'), enterprise_val=ev_for_ratios, current_price_val=current_price, shares_outstanding_val=shares_outstanding, currency_sym=currency_symbol, user_benchmark_dict=user_benchmarks, benchmark_source_dict=benchmark_sources, wacc_data=wacc_components)
    except Exception as e: display(Markdown(f"❌ **CRITICAL ERROR Ratio Analysis: {e}**")); print(traceback.format_exc()); ratios_calculated = {'error': f'Runtime error: {e}'}
else:
    reason = [];
    if not required_statements_present: missing_stmts = [];
    if not financials_present: missing_stmts.append("Income Statement");
    if not balance_sheet_present: missing_stmts.append("Balance Sheet"); reason.append(f"Missing essential statement(s): {', '.join(missing_stmts)}")
    if not valuation_ok_for_ratios: reason.append(f"Valuation failed fatally ({final_valuation.get('error', 'Unknown')})")
    if not wacc_components_exist: reason.append("WACC components calculation failed")
    error_reason = "; ".join(reason) if reason else "Unknown upstream errors"; display(Markdown(f"❌ **Skipping Ratio Analysis:** {error_reason}")); ratios_calculated = {'error': f'Skipped: {error_reason}'}


# %% [markdown]
# ## 10. Summary & Visualization Dashboard
# (Section Unchanged)

# %%
# Dashboard Function Definition
def display_summary_dashboard(company_nm, ticker_sym, current_pr, intrinsic_val_per_sh,
                              wacc_val, term_growth, init_growth, growth_src,
                              final_val_dict, hist_df, fcff_res,
                              currency_sym, info_dict, input_src_dict):

    display(Markdown(f"## Valuation Summary Dashboard: {company_nm} ({ticker_sym})"))
    sources = []; key_map = {'Rf':'Rf', 'Rm':'Rm', 'Beta':'Beta', 'Credit Spread':'Spread', 'Benchmarks':'Benchmarks', 'FCFF Method':'FCFF Method'}
    for key, mapped_key in key_map.items(): source_val = input_src_dict.get(key, '?');
    if isinstance(source_val, str): source_clean = source_val.split('(')[0].strip().replace('User Input/Default', 'User/Def').replace('User Input', 'User').replace('Dynamic Fetch', 'Dyn').replace('Default','Def').replace('Derived','Deriv'); sources.append(f"{mapped_key}:`{source_clean}`")
    source_summary = " | ".join(sources)
    growth_disp = growth_src if isinstance(growth_src,str) else 'N/A'; growth_disp_clean = growth_disp.split('[')[0].strip().replace('Geometric Mean','GM').replace('valid CAGR(s)','')
    fcff_basis_disp = fcff_res.get('fcff_basis_method', 'N/A') if isinstance(fcff_res, dict) else 'N/A'; fcff_basis_clean = fcff_basis_disp.split('(User Selected')[0].strip() if isinstance(fcff_basis_disp, str) else 'N/A'
    display(Markdown(f"**Key Inputs:** {source_summary} | **Init Growth:** `{growth_disp_clean}` | **FCFF Basis:** `{fcff_basis_clean}`"))

    valid_iv = intrinsic_val_per_sh is not None and np.isfinite(intrinsic_val_per_sh); valid_cp = current_pr is not None and np.isfinite(current_pr) and current_pr > 0; valid_w = wacc_val is not None and np.isfinite(wacc_val); valid_tg = term_growth is not None and np.isfinite(term_growth); valid_ig = init_growth is not None and np.isfinite(init_growth)
    fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]], subplot_titles=("Price vs. Intrinsic Value", "WACC", "Growth Rates"), horizontal_spacing=0.15)
    iv_num = intrinsic_val_per_sh if valid_iv else np.nan; iv_fmt = format_currency(iv_num) if valid_iv else "N/A"; cp_num = current_pr if valid_cp else 0.0; delta_config = None; comparison_text = f"Intrinsic Value: {iv_fmt}"; mode = "number"
    if valid_iv and valid_cp: mode = "number+delta";
    if iv_num > 1e-9: upside = (iv_num / cp_num) - 1; comparison_text += f" (Upside: {upside:+.1%})"; delta_config = {'reference': iv_num, 'relative': False, 'valueformat': '.2f', 'prefix': currency_sym, 'decreasing': {'color': '#2ecc71'}, 'increasing': {'color': '#e74c3c'}}
    else: comparison_text += " (≤ 0)"; delta_config = {'reference': 0, 'relative': False, 'valueformat': '.2f', 'prefix': currency_sym, 'increasing': {'color': '#e74c3c'}}
    elif valid_iv: comparison_text += " (Current N/A)"
    else: comparison_text = f"Intrinsic Value: N/A (Error)"
    fig.add_trace(go.Indicator(mode=mode, value=cp_num, number={'prefix': currency_sym, 'valueformat': '.2f'}, title={"text": f"<span style='font-size:0.8em'>{comparison_text}</span>", 'font_size': 12}, delta=delta_config, domain={'row': 0, 'column': 0}), row=1, col=1)
    wacc_num = wacc_val * 100 if valid_w else None; gauge_config_wacc = None
    if valid_w: gauge_config_wacc = {'axis': {'range': [MIN_WACC * 100, MAX_WACC * 100], 'tickformat': '.1f'}, 'bar': {'color': "darkblue"}, 'steps': [{'range': [MIN_WACC * 100, 9], 'color': "#d5f5e3"}, {'range': [9, 14], 'color': "#fdebd0"}, {'range': [14, MAX_WACC * 100], 'color': "#fadbd8"}], 'threshold': {'line': {'color': "red", 'width': 2}, 'thickness': 0.75, 'value': wacc_num}}
    fig.add_trace(go.Indicator(mode="gauge+number", value=wacc_num, number={'suffix': "%", 'valueformat': '.1f' if valid_w else ''}, gauge=gauge_config_wacc, title={"text": "<span style='font-size:0.8em'>WACC</span>", 'font_size': 12}, domain={'row': 0, 'column': 1}), row=1, col=2)
    tg_num = term_growth * 100 if valid_tg else None; ig_fmt = format_percent(init_growth, 1) if valid_ig else "N/A"; g_title = f"<span style='font-size:0.8em'>Term Growth (g)<br>(Initial: {ig_fmt})</span>"; gauge_config_growth = None
    if valid_tg: g_range_min = min(MIN_GROWTH * 100, tg_num - 2); g_range_max = max(TERMINAL_GROWTH_RATE_CAP * 100 + 1, tg_num + 2); gauge_config_growth = {'axis': {'range': [g_range_min, g_range_max], 'tickformat': '.1f'}, 'bar': {'color': "darkgrey"}, 'steps': [{'range': [g_range_min, 2.0], 'color': "#fadbd8"}, {'range': [2.0, 4.5], 'color': "#fdebd0"}, {'range': [4.5, g_range_max], 'color': "#d5f5e3"}], 'threshold': {'line': {'color': "black", 'width': 2}, 'thickness': 0.75, 'value': tg_num}}
    fig.add_trace(go.Indicator(mode="gauge+number", value=tg_num, number={'suffix': "%", 'valueformat': '.1f' if valid_tg else ''}, gauge=gauge_config_growth, title={"text": g_title, 'font_size': 12}, domain={'row': 0, 'column': 2}), row=1, col=3)
    fig.update_layout(height=250, margin=dict(l=30, r=30, t=60, b=20)); fig.show()

    if hist_df is not None and not hist_df.empty:
        price_col = None;
        if 'Adj Close' in hist_df.columns and hist_df['Adj Close'].notna().any(): price_col = 'Adj Close'
        elif 'Close' in hist_df.columns and hist_df['Close'].notna().any(): price_col = 'Close'
        if price_col:
             fig_h = px.line(hist_df, x=hist_df.index, y=price_col, title=f'{company_nm} Price History ({price_col}) vs. DCF', labels={'x': 'Date', price_col: f'Price ({currency_sym})'}); fig_h.update_layout(height=400, showlegend=False); fig_h.update_xaxes(rangeslider_visible=False)
             if valid_iv: fig_h.add_hline(y=iv_num, line_dash="dash", line_color="#e74c3c", annotation_text=f"DCF: {format_currency(iv_num)}", annotation_position="bottom right", annotation_font_color="#e74c3c")
             if valid_cp: fig_h.add_hline(y=cp_num, line_dash="dot", line_color="#3498db", annotation_text=f"Current: {format_currency(cp_num)}", annotation_position="top left", annotation_font_color="#3498db")
             fig_h.show()
        else: display(Markdown("*Hist price col N/A.*"))
    else: display(Markdown("*Hist data N/A.*"))

    fcff_data_valid_for_pie = isinstance(fcff_res, dict); fcff_fatal_pie = fcff_data_valid_for_pie and fcff_res.get('error') and ("halted" in fcff_res.get('error', '') or "Cannot proceed" in fcff_res.get('error', ''))
    if fcff_data_valid_for_pie and not fcff_fatal_pie:
        pv_f = fcff_res.get('cumulative_pv_fcff', np.nan); pv_t = fcff_res.get('pv_terminal_value', np.nan); valid_pvs = [v for v in [pv_f, pv_t] if np.isfinite(v)]; total_ev = sum(valid_pvs) if valid_pvs else np.nan
        if np.isfinite(total_ev):
            labels=[]; values=[]; pulls=[]; colors=['#4e79a7','#f28e2b']
            if abs(pv_f) > 1e-6: labels.append(f'PV({FORECAST_YEARS}yr FCFFs)'); values.append(abs(pv_f)); pulls.append(0)
            if abs(pv_t) > 1e-6: labels.append('PV(TV)'); values.append(abs(pv_t)); pulls.append(0.05 if len(labels) > 1 else 0)
            if labels: hover_template = f"<b>%{{label}}</b><br>Value Contrib:{currency_sym} %{{value:,.0f}}<br>%{{percent}}<extra></extra>"; fig_pie = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4, pull=pulls, marker_colors=colors[:len(labels)], hovertemplate=hover_template, textinfo='percent+label', sort=False, rotation=90)]); fig_pie.update_layout(title_text=f"DCF EV Breakdown (Total: {format_currency(total_ev)})", title_x=0.5, height=350, showlegend=False, margin=dict(t=60, b=30, l=30, r=30)); fig_pie.show()
            else: display(Markdown(f"*DCF components zero/neg.*"))
        elif fcff_fatal_pie: display(Markdown(f"*Cannot gen EV chart: Fatal FCFF error.*"))
        else: display(Markdown(f"*EV calc failed ({format_currency(total_ev)}).*"))
    else: display(Markdown(f"*FCFF results invalid.*"))

    fcff_proj_data_valid = isinstance(fcff_res, dict) and not fcff_fatal_pie
    if fcff_proj_data_valid:
        proj_fcffs = fcff_res.get('projected_fcffs', []); pv_proj_fcffs = fcff_res.get('pv_fcffs', []); num_proj = len(proj_fcffs)
        if num_proj > 0 and len(pv_proj_fcffs) == num_proj:
            years = [f'Year {i+1}' for i in range(num_proj)]; fcff_clean = [f if np.isfinite(f) else 0 for f in proj_fcffs]; pv_clean = [p if np.isfinite(p) else 0 for p in pv_proj_fcffs]
            df_proj = pd.DataFrame({'Year': years, 'Projected FCFF': fcff_clean, 'PV of FCFF': pv_clean}); df_melt = df_proj.melt(id_vars='Year', var_name='Cash Flow Type', value_name='Value')
            fig_bar = px.bar(df_melt, x='Year', y='Value', color='Cash Flow Type', title=f'Projected FCFF & PV ({FORECAST_YEARS} Yrs)', labels={'Value':f'Value ({currency_sym})', 'Year': 'Forecast Year'}, barmode='group', height=350, color_discrete_map={'Projected FCFF': '#5dade2', 'PV of FCFF': '#af7ac5'})
            hover_template_bar = f"<b>%{{customdata[0]}}</b><br>Yr:%{{x}}<br>Val:{currency_sym} %{{y:,.0f}}<extra></extra>"; fig_bar.update_traces(hovertemplate=hover_template_bar, customdata=df_melt[['Cash Flow Type']]); fig_bar.update_layout(yaxis_title=f"Value ({currency_sym})", legend_title="Cash Flow Type"); fig_bar.show()
        elif num_proj == 0: display(Markdown("*No projected FCFF data.*"))
        else: display(Markdown("*FCFF data mismatch.*"))
    elif fcff_fatal_pie: display(Markdown(f"*Cannot gen FCFF chart: Fatal error.*"))
    else: display(Markdown(f"*FCFF data N/A.*"))

    display(Markdown("### Valuation Conclusion")); conclusion = ""
    if isinstance(final_val_dict, dict):
        iv = final_val_dict.get('intrinsic_value_per_share', np.nan); v_err = final_val_dict.get('error'); f_err = fcff_res.get('error') if isinstance(fcff_res,dict) else "FCFF Invalid"
        if v_err and "halted" in v_err: conclusion = f"<span style='color:red; font-weight:bold;'>❌ Val Halted: {v_err}</span>"
        elif fcff_fatal_pie: conclusion = f"<span style='color:red; font-weight:bold;'>❌ Val Halted (FCFF Error): {f_err}</span>"
        elif v_err: conclusion = f"<span style='color:red; font-weight:bold;'>❌ Val Error: {v_err}</span>"
        elif not valid_iv: conclusion=f"<span style='color:red; font-weight:bold;'>❌ Val Error: IV calc failed.</span>"
        elif valid_iv and iv <= 0: conclusion=f"<span style='color:orange; font-weight:bold;'>⚠️ Neg/Zero DCF Value: {format_currency(iv)}.</span>"
        elif valid_iv and not valid_cp: conclusion=f"✅ DCF Val: **{format_currency(iv)}**. Cannot compare."
        elif valid_iv and valid_cp:
             diff = (iv / cp_num) - 1; cp_fmt = format_currency(cp_num); iv_fmt = format_currency(iv); margin = 0.15
             if abs(diff) < margin: conclusion = f"<span style='color:grey; font-weight:bold;'>↔️ Fairly Valued (~+/-{margin:.0%}):</span> Price ({cp_fmt}) vs DCF ({iv_fmt}). Diff:**{diff:+.1%}**"
             elif diff < 0: over_pct = (cp_num / iv) - 1 if iv != 0 else float('inf'); conclusion = f"<span style='color:#e74c3c; font-weight:bold;'>🔻 Potentially Overvalued:</span> Price ({cp_fmt}) **{over_pct:+.1%}** > DCF ({iv_fmt})."
             else: conclusion = f"<span style='color:#2ecc71; font-weight:bold;'>✅ Potentially Undervalued:</span> Price ({cp_fmt}) **{diff:+.1%}** < DCF ({iv_fmt})."
        else: conclusion=f"<span style='color:red; font-weight:bold;'>❌ Val Error: Unexpected.</span>"
    else: conclusion=f"<span style='color:red; font-weight:bold;'>❌ Val Error: Data invalid.</span>"
    display(Markdown(f"> **{conclusion}**"))

# --- Execute Summary Dashboard Generation ---
valuation_successful = (isinstance(final_valuation, dict) and not (final_valuation.get('error') and "halted" in final_valuation.get('error','')))
fcff_data_exists = isinstance(fcff_results, dict); info_data_exists = isinstance(stock_data.get('info'), dict); input_sources_exist = isinstance(input_sources, dict); history_data_exists = stock_data.get('history') is not None
if valuation_successful and fcff_data_exists and info_data_exists and input_sources_exist and history_data_exists:
    try: display_summary_dashboard(company_nm=company_name, ticker_sym=ticker_symbol, current_pr=current_price, intrinsic_val_per_sh=final_valuation.get('intrinsic_value_per_share'), wacc_val=discount_rate, term_growth=fcff_results.get('terminal_growth_rate'), init_growth=fcff_results.get('initial_growth_rate'), growth_src=fcff_results.get('growth_source', 'N/A'), final_val_dict=final_valuation, hist_df=stock_data['history'], fcff_res=fcff_results, currency_sym=currency_symbol, info_dict=stock_data['info'], input_src_dict=input_sources)
    except Exception as e: display(Markdown(f"❌ **CRITICAL ERROR Dashboard: {e}**")); print(traceback.format_exc())
else:
    skip_reasons = [];
    if not valuation_successful: skip_reasons.append(f"Valuation failed/halted ({final_valuation.get('error', 'Unknown')})")
    if not fcff_data_exists: skip_reasons.append("FCFF results missing")
    if not info_data_exists: skip_reasons.append("Stock info missing")
    if not input_sources_exist: skip_reasons.append("Input sources missing")
    if not history_data_exists: skip_reasons.append("History data missing")
    skip_reason_str = ", ".join(skip_reasons) if skip_reasons else "Unknown reason"; display(Markdown(f"❌ **Skipping Summary Dashboard:** {skip_reason_str}"));


# %% [markdown]
# ## 11. Assumptions & Disclaimer (Reflects V21.7 Logic)

# %%
# --- Dynamically Generate Assumption Strings from Results ---
rf_src = input_sources.get('Rf','?')
rm_src = input_sources.get('Rm','?')
beta_src = input_sources.get('Beta','?')
cs_src = input_sources.get('Credit Spread','N/A')
bench_src = input_sources.get('Benchmarks','?')
fcff_method_src = input_sources.get('FCFF Method','N/A')
growth_src = fcff_results.get('growth_source','N/A') if isinstance(fcff_results,dict) else 'Error'
term_g = fcff_results.get('terminal_growth_rate',np.nan) if isinstance(fcff_results,dict) else np.nan
init_g = fcff_results.get('initial_growth_rate',np.nan) if isinstance(fcff_results,dict) else np.nan
basis_meth = fcff_results.get('fcff_basis_method','N/A') if isinstance(fcff_results,dict) else 'Error'
tax_src = wacc_components.get('tax_rate_source','N/A') if isinstance(wacc_components,dict) else 'Error'
eq_src = wacc_components.get('equity_source','N/A') if isinstance(wacc_components,dict) else 'Error'
wgt_src = wacc_components.get('weights_source','N/A') if isinstance(wacc_components,dict) else 'Error'
kd_method = wacc_components.get('debt_calc_method','N/A') if isinstance(wacc_components,dict) else 'Error'
term_g_fmt = format_percent(term_g) if np.isfinite(term_g) else "N/A"
init_g_fmt = format_percent(init_g) if np.isfinite(init_g) else "N/A"
def_init_g_fmt = format_percent(DEFAULT_GROWTH_RATE)
min_init_g_fmt = format_percent(MIN_INITIAL_GROWTH_FLOOR)
max_init_g_fmt = format_percent(MAX_INITIAL_GROWTH_CEILING)
def clean_source(source_str):
    if not isinstance(source_str, str): return str(source_str)
    return source_str.split('(')[0].strip().replace('User Input/Default', 'User/Def').replace('User Input', 'User').replace('Dynamic Fetch', 'Dynamic').replace('Default','Default').replace('Derived','Derived').replace('Fallback ','')

# %%
# ----> Display Assumptions & Disclaimer (Updated for V21.7 FCFF Methods) <----
display(Markdown(f"""
**Key Assumptions & Input Sources:**

*   **Financial Data:** Sourced primarily from Yahoo Finance via `yfinance`. Accuracy/availability subject to provider. Quarterly used as fallback.
*   **Number Formatting:** Uses Indian Lakh/Crore system (₹).
*   **Risk-Free Rate (Rf):** `{clean_source(rf_src)}`. Dynamic fetch attempted (`{rf_proxy_ticker}`).
*   **Expected Market Return (Rm):** `{clean_source(rm_src)}`.
*   **Market Risk Premium (MRP):** Derived (`Rm` - `Rf`).
*   **Beta (β):** `{clean_source(beta_src)}`.
*   **Cost of Debt (Kd):** `{kd_method}` method. Bounded [{format_percent(MIN_COST_OF_DEBT)}, {format_percent(MAX_COST_OF_DEBT)}].
*   **Credit Spread:** `{clean_source(cs_src)}` (if default Kd used).
*   **Tax Rate:** `{tax_src}`. Bounded [{format_percent(MIN_TAX_RATE)}, {format_percent(MAX_TAX_RATE)}].
*   **FCFF Calculation Method:**
    *   4-year average FCFF pre-calculated for 5 methods (0-4 below).
    *   User selected based on positive averages (`Selected: {clean_source(fcff_method_src)}`).
    *   Basis for projection is the 4-yr avg of the selected method (`{basis_meth}`).
    *   Historical FCFF for CAGR calculation uses *only* the selected method.
    *   *(Note: Method 6 (UNI-based) omitted as redundant to Method 3. Method 7 (Forecast-based) not applicable for historical averaging).*
    *   **Projection halts if the calculated 4-year average basis is not positive.**
*   **Growth Rates:**
    *   **Initial Growth:** **`{init_g_fmt}`**. (`Source: {growth_src}`).
        *   Method: Geometric Mean of 3-Year Revenue CAGR & 3-Year FCFF CAGR (using selected method's history). **Stock CAGR NOT used.**
        *   Fallback: Defaults to `{def_init_g_fmt}`.
        *   Bounds: [`{min_init_g_fmt}`, `{max_init_g_fmt}`].
    *   **Terminal Growth (g):** **`{term_g_fmt}`**. Min(Config Cap, WACC*0.6, Initial Growth, Rf*0.9). Floored & `< WACC`.
*   **Projection Period:** FCFF projected for **{FORECAST_YEARS} years**. Linear growth decay.
*   **Capital Structure:** Weights based on Equity (`{clean_source(eq_src)}`) & Debt (`{kd_method}`). (`Weights Source: {clean_source(wgt_src)}`)
*   **Industry Benchmarks:** From `{clean_source(bench_src)}`.

**Limitations & Considerations:**

*   **Model Sensitivity:** DCF highly sensitive to inputs (WACC, growth, **chosen FCFF method**).
*   **FCFF Method Choice Impact:** Choice significantly alters valuation. See formulas used. Method 4 (EBITDA Proxy) is approximate.
*   **Dependency on Positive Basis:** Valuation halts if chosen method yields non-positive avg basis.
*   **Input Quality:** Validity depends on user inputs/fallbacks.
*   **Historical Data:** Limited data impacts CAGR/average reliability.
*   **CAGR/GM Limitations:** Standard limitations apply. GM only uses Rev/FCFF.
*   **Dynamic Data Reliability:** Depends on `yfinance`.
*   **Simplifications:** Linear decay, constant rates/structure. Ignores qualitative factors.
*   **Data Completeness:** Performance degrades with missing data.

**Disclaimer:**
*For **educational purposes only**. **NOT investment advice**. Conduct own due diligence. Consult qualified advisor. Creators NOT liable.*
"""))




SyntaxError: invalid syntax (1206085326.py, line 1490)